In [4]:
import requests
headers = {
    "X-RapidAPI-Key": "3eaece787amshfe1adf78b14d67bp1c781fjsnebfd830416ef",
    "X-RapidAPI-Host": "cricbuzz-cricket.p.rapidapi.com"
}

BASE_URL = "https://cricbuzz-cricket.p.rapidapi.com"

In [5]:
def team_players_india():
    team_id = 2 #India
    from utils.setup_database import init_db
    conn, cursor = init_db()
    def table_creation():
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS players (
            player_id INTEGER PRIMARY KEY,
            full_name TEXT,
            role TEXT,
            batting_style TEXT,
            bowling_style TEXT,
            team_name TEXT
        )
        """)
        conn.commit()
    def set_db(player):
        cursor.execute("""
        INSERT OR REPLACE INTO players 
        (player_id, full_name, role, batting_style, bowling_style, team_name)
         VALUES (?, ?, ?, ?, ?, ?)
            """, (
                player.get("id"),
                player.get("name"),
                player.get("role"),
                player.get("batting_style"),
                player.get("bowling_style"),
                "India"
            ))
        conn.commit()
        print("✅ done")

    def get_team_players(team_id):
        url =f"{BASE_URL}/teams/v1/{team_id}/players"
        response = requests.get(url, headers=headers)
        data = response.json()
        players_ids =[]
        for players in data.get('player'):
            players_ids.append(players.get('id'))
        return players_ids
    def getplayerdetails(player_id):
        url = f"{BASE_URL}/stats/v1/player/{player_id}"
        response = requests.get(url, headers=headers)
        data = response.json()
        details ={
            "id":data.get('id'),
            "name":data.get('name'),
            "role":data.get('role'),
            "batting_style":data.get('bat'),
            "bowling_style":data.get('bowl')

        }
        return details
    
    ids = get_team_players(team_id)
    player_details =[]
    for id in ids:
        if not id :
            continue
        else:
            player_details.append(getplayerdetails(id))

    table_creation()
    for player in player_details:
        set_db(player)


team_players_india()

✅ done
✅ done
✅ done
✅ done
✅ done
✅ done
✅ done
✅ done
✅ done
✅ done
✅ done
✅ done
✅ done
✅ done
✅ done
✅ done
✅ done
✅ done
✅ done
✅ done
✅ done
✅ done
✅ done
✅ done
✅ done
✅ done
✅ done
✅ done
✅ done
✅ done
✅ done
✅ done
✅ done
✅ done
✅ done
✅ done
✅ done


In [ ]:
import requests
import sqlite3
import time

# -----------------------
# RapidAPI Setup
# -----------------------
HEADERS = {
    "X-RapidAPI-Key": "3eaece787amshfe1adf78b14d67bp1c781fjsnebfd830416ef",
    "X-RapidAPI-Host": "cricbuzz-cricket.p.rapidapi.com"
}

BASE_URL = "https://cricbuzz-cricket.p.rapidapi.com"

# -----------------------
# Database Setup
# -----------------------
conn = sqlite3.connect("cricket_fresh.db")
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS players (
    player_id INTEGER PRIMARY KEY,
    full_name TEXT,
    role TEXT,
    bat_style TEXT,
    bowl_style TEXT,
    team_name TEXT
)
""")

# -----------------------
# Step 1: Get India Team Players
# -----------------------
team_id = 2  # India (usually 2 in Cricbuzz)
url =f"{BASE_URL}/teams/v1/{team_id}/players"
resp = requests.get(url, headers=HEADERS)

if resp.status_code == 200:
    players_data = resp.json()

    # -----------------------
    # Step 2: Loop through players and fetch details
    # -----------------------
    for p in players_data.get("player", []):  # API returns player list
        player_id = p.get("id")
        if not player_id:
            continue

        player_url = f"{BASE_URL}/players/v1/player/{player_id}"
        player_resp = requests.get(player_url, headers=HEADERS)

        if player_resp.status_code == 200:
            details = player_resp.json()
            
            full_name = details.get("name")
            role = details.get("role")
            bat_style = details.get("bat")
            bowl_style = details.get("bowl")
            team_name = details.get("teamName")

            # Insert into DB
            cursor.execute("""
            INSERT OR REPLACE INTO players (player_id, full_name, role, bat_style, bowl_style, team_name)
            VALUES (?, ?, ?, ?, ?, ?)
            """, (player_id, full_name, role, bat_style, bowl_style, team_name))

            conn.commit()
            print(f"✅ Inserted: {full_name}")

        time.sleep(1)  # avoid rate limiting
else:
    print("❌ Error fetching team players:", resp.text)

conn.close()


In [2]:
import requests
import mysql.connector
import time

# -----------------------
# RapidAPI Setup
# -----------------------
HEADERS = {
    "X-RapidAPI-Key": "3eaece787amshfe1adf78b14d67bp1c781fjsnebfd830416ef",
    "X-RapidAPI-Host": "cricbuzz-cricket.p.rapidapi.com"
}

BASE_URL = "https://cricbuzz-cricket.p.rapidapi.com"

# -----------------------
# MySQL Setup
# -----------------------
conn = mysql.connector.connect(
    host="localhost",       # your MySQL host
    user="root",            # your MySQL user
    password="2741",   # your MySQL password
    database="cricketdb"   # your database
)
cursor = conn.cursor()

# Create table if not exists
cursor.execute("""
CREATE TABLE IF NOT EXISTS players (
    player_id INT PRIMARY KEY,
    full_name VARCHAR(255),
    role VARCHAR(50),
    bat_style VARCHAR(50),
    bowl_style VARCHAR(50),
    team_name VARCHAR(50)
)
""")

# -----------------------
# Step 1: Get India Team Players
# -----------------------
team_id = 2  # India
url = f"{BASE_URL}/teams/v1/{team_id}/players"
resp = requests.get(url, headers=HEADERS)

if resp.status_code == 200:
    players_data = resp.json()

    # -----------------------
    # Step 2: Loop through players and fetch details
    # -----------------------
    for p in players_data.get("player", []):
        player_id = p.get("id")
        if not player_id:
            continue

        player_url = f"{BASE_URL}/stats/v1/player/{player_id}"  # profile endpoint
        player_resp = requests.get(player_url, headers=HEADERS)

        if player_resp.status_code == 200:
            details = player_resp.json()

            full_name = details.get("name")        # note: MySQL column expects string
            role = details.get("role")
            bat_style = details.get("bat")
            bowl_style = details.get("bowl")
            team_name = "India"

            # Insert into MySQL
            cursor.execute("""
            INSERT INTO players (player_id, full_name, role, bat_style, bowl_style, team_name)
            VALUES (%s, %s, %s, %s, %s, %s)
            ON DUPLICATE KEY UPDATE
                full_name = VALUES(full_name),
                role = VALUES(role),
                bat_style = VALUES(bat_style),
                bowl_style = VALUES(bowl_style),
                team_name = VALUES(team_name)
            """, (player_id, full_name, role, bat_style, bowl_style, team_name))

            conn.commit()
            print(f"✅ Inserted/Updated: {full_name}")

        time.sleep(1)  # avoid rate limit
else:
    print("❌ Error fetching team players:", resp.text)

conn.close()


✅ Inserted/Updated: Shubman Gill
✅ Inserted/Updated: Yashasvi Jaiswal
✅ Inserted/Updated: Sai Sudharsan
✅ Inserted/Updated: Rohit Sharma
✅ Inserted/Updated: Virat Kohli
✅ Inserted/Updated: Suryakumar Yadav
✅ Inserted/Updated: Shreyas Iyer
✅ Inserted/Updated: Rinku Singh
✅ Inserted/Updated: Tilak Varma
✅ Inserted/Updated: Ruturaj Gaikwad
✅ Inserted/Updated: Rajat Patidar
✅ Inserted/Updated: Sarfaraz Khan
✅ Inserted/Updated: Abhimanyu Easwaran
✅ Inserted/Updated: Karun Nair
✅ Inserted/Updated: Nitish Kumar Reddy
✅ Inserted/Updated: Hardik Pandya
✅ Inserted/Updated: Shivam Dube
✅ Inserted/Updated: Abhishek Sharma
✅ Inserted/Updated: Ravindra Jadeja
✅ Inserted/Updated: Shardul Thakur
✅ Inserted/Updated: Washington Sundar
✅ Inserted/Updated: Axar Patel
✅ Inserted/Updated: Rishabh Pant
✅ Inserted/Updated: KL Rahul
✅ Inserted/Updated: Dhruv Jurel
✅ Inserted/Updated: Sanju Samson
✅ Inserted/Updated: Ishan Kishan
✅ Inserted/Updated: Mohammed Siraj
✅ Inserted/Updated: Jasprit Bumrah
✅ Inserted/U

In [6]:
import requests
import mysql.connector

HEADERS = {
    "X-RapidAPI-Key": "3eaece787amshfe1adf78b14d67bp1c781fjsnebfd830416ef",
    "X-RapidAPI-Host": "cricbuzz-cricket.p.rapidapi.com"
}

# MySQL connection
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="2741",
    database="cricketdb"
)
cursor = conn.cursor()

# Fetch teams from API
url = "https://cricbuzz-cricket.p.rapidapi.com/teams/v1/international"
resp = requests.get(url, headers=HEADERS)

if resp.status_code == 200:
    teams_data = resp.json()
    for t in teams_data.get("list", []):

        team_id = t.get("teamId")
        team_name = t.get("teamName")
        country = t.get("teamName")
        if team_id:
            cursor.execute("""
            INSERT INTO teams (team_id, team_name, country)
            VALUES (%s, %s, %s)
            ON DUPLICATE KEY UPDATE team_name=VALUES(team_name), country=VALUES(country)
        """, (team_id, team_name, country))
        

    conn.commit()
    print("✅ Teams table populated from API.")
else:
    print("❌ Error fetching teams:", resp.text)

conn.close()


✅ Teams table populated from API.


In [ ]:
import requests
import mysql.connector

# -------------------
# RapidAPI Config
# -------------------
headers = {
    "X-RapidAPI-Key": "3eaece787amshfe1adf78b14d67bp1c781fjsnebfd830416ef",
    "X-RapidAPI-Host": "cricbuzz-cricket.p.rapidapi.com"
}


# -------------------
# Database Connection
# -------------------
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="2741",
    database="cricketdb"
)
cursor = conn.cursor()


# -------------------
# Create Table
# -------------------


# -------------------
# Insert Data
# -------------------

# url = "https://cricbuzz-cricket.p.rapidapi.com/teams/v1/international"
# response = requests.get(url, headers=headers)
# data = response.json()
# teams_ids = []
# player_ids = []

# for da in data.get('list', []):
#     tid = da.get('teamId')  # returns None if key doesn't exist
#     if tid is not None:
#         teams_ids.append(tid)
#         player_teams_url = f"https://cricbuzz-cricket.p.rapidapi.com/teams/v1/{tid}/players"
#         response1 = requests.get(player_teams_url, headers=headers)
#         data1 = response1.json()
#         for playr in data1['player']:
#             pid = playr.get('id')
#             player_id = playr.get("id")
#             player_name = playr.get("name")
#             batting_style = playr.get("bat")
#             bowling_style = playr.get("bowl")
#             if pid is not None:
#                 player_ids.append(pid)
#                 cursor.execute("""
#                         INSERT INTO players (player_id, player_name, batting_style, bowling_style, team_id)
#                         VALUES (%s, %s, %s, %s, %s)
#                         ON DUPLICATE KEY UPDATE
#                             player_name = VALUES(player_name),
#                             batting_style = VALUES(batting_style),
#                             bowling_style = VALUES(bowling_style),
#                             team_id = VALUES(team_id)
#                     """, (player_id, player_name, batting_style, bowling_style, tid))

#                 # player_url =f"https://cricbuzz-cricket.p.rapidapi.com/stats/v1/player/{pid}"
#                 # player_response = requests.get(player_url,headers=headers)
#                 # player =player_response.json()
#                 # role = player.get("role")
#                 # cursor.execute("""
#                 #     INSERT INTO players (player_id, player_name, role, batting_style, bowling_style, team_id)
#                 #     VALUES (%s, %s, %s, %s, %s, %s)
#                 #     ON DUPLICATE KEY UPDATE
#                 #         player_id = VALUES(player_id),
#                 #         player_name = VALUES(player_name),
#                 #         role = VALUES(role),
#                 #         batting_style = VALUES(batting_style),
#                 #         bowling_style = VALUES(bowling_style),
#                 #         team_id = VALUES(team_id)
#                 #     """,(player_id,player_name,role,batting_style,bowling_style,tid))

# conn.commit()
# print("✅ Players inserted successfully!")



In [2]:
import requests
url = "https://cricbuzz-cricket.p.rapidapi.com/teams/v1/international"
headers = {
    "X-RapidAPI-Key": "3eaece787amshfe1adf78b14d67bp1c781fjsnebfd830416ef",
    "X-RapidAPI-Host": "cricbuzz-cricket.p.rapidapi.com"
}
response = requests.get(url, headers=headers)
data = response.json()
teams_ids = []
player_ids = []

for da in data.get('list', []):
    tid = da.get('teamId')  # returns None if key doesn't exist
    if tid is not None:
        teams_ids.append(tid)

In [3]:
import requests
for tid in teams_ids: 
    url = f"https://cricbuzz-cricket.p.rapidapi.com/teams/v1/{tid}/players"
    headers = {
	"x-rapidapi-key": "3eaece787amshfe1adf78b14d67bp1c781fjsnebfd830416ef",
	"x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        #try:
        data1 = response.json()
        players = data1.get("player", [])
        # print(players)
        for playr in players:
           if "id" in playr:  # skip category headers like "BATSMEN"
                    pid = playr["id"]
                    player_url =f"https://cricbuzz-cricket.p.rapidapi.com/stats/v1/player/{pid}"
                    player_response = requests.get(player_url,headers=headers)
                    player =player_response.json()
                    player_id = player.get('id')
                    role = player.get("role")
                    player_name = player.get("name")
                    batting_style = player.get("bat")
                    bowling_style = player.get("bowl")
                    cursor.execute("""
                        INSERT INTO Players (player_id, player_name, role, batting_style, bowling_style, team_id)
                        VALUES (%s, %s, %s, %s, %s, %s)
                        ON DUPLICATE KEY UPDATE
                            player_id = VALUES(player_id),
                            player_name = VALUES(player_name),
                            role = VALUES(role),
                            batting_style = VALUES(batting_style),
                            bowling_style = VALUES(bowling_style),
                            team_id = VALUES(team_id)
                        """,(player_id,player_name,role,batting_style,bowling_style,tid))
                    
                    conn.commit()
                    print("✅ Inserted:", player_name)

                    

    

✅ Inserted: Shubman Gill
✅ Inserted: Yashasvi Jaiswal
✅ Inserted: Sai Sudharsan
✅ Inserted: Rohit Sharma
✅ Inserted: Virat Kohli
✅ Inserted: Suryakumar Yadav
✅ Inserted: Shreyas Iyer
✅ Inserted: Rinku Singh
✅ Inserted: Tilak Varma
✅ Inserted: Ruturaj Gaikwad
✅ Inserted: Rajat Patidar
✅ Inserted: Sarfaraz Khan
✅ Inserted: Abhimanyu Easwaran
✅ Inserted: Karun Nair
✅ Inserted: Nitish Kumar Reddy
✅ Inserted: Hardik Pandya
✅ Inserted: Shivam Dube
✅ Inserted: Abhishek Sharma
✅ Inserted: Ravindra Jadeja
✅ Inserted: Shardul Thakur
✅ Inserted: Washington Sundar
✅ Inserted: Axar Patel
✅ Inserted: Rishabh Pant
✅ Inserted: KL Rahul
✅ Inserted: Dhruv Jurel
✅ Inserted: Sanju Samson
✅ Inserted: Ishan Kishan
✅ Inserted: Mohammed Siraj
✅ Inserted: Jasprit Bumrah
✅ Inserted: Prasidh Krishna
✅ Inserted: Akash Deep
✅ Inserted: Kuldeep Yadav
✅ Inserted: Arshdeep Singh
✅ Inserted: Harshit Rana
✅ Inserted: Ravi Bishnoi
✅ Inserted: Mukesh Kumar
✅ Inserted: Varun Chakaravarthy
✅ Inserted: Hashmatullah Shahidi


: 

In [ ]:
# MySQL Setup
import mysql.connector
# -----------------------
conn = mysql.connector.connect(
    host="localhost",       # your MySQL host
    user="root",            # your MySQL user
    password="2741",   # your MySQL password
    database="cricketdb"   # your database
)
cursor = conn.cursor()
cursor.execute("""
CREATE TABLE IF NOT EXISTS Players (
    player_id INT PRIMARY KEY,
    player_name VARCHAR(150),
    role VARCHAR(100),
    batting_style VARCHAR(100),
    bowling_style VARCHAR(100),
    team_id INT,
    FOREIGN KEY (team_id) REFERENCES teams(team_id)
)
""")


In [ ]:
# MySQL Setup
import mysql.connector
# -----------------------
conn = mysql.connector.connect(
    host="localhost",       # your MySQL host
    user="root",            # your MySQL user
    password="2741",   # your MySQL password
    database="cricketdb"   # your database
)
cursor = conn.cursor()
# -----------------------------
# 3. venues table
# -----------------------------
cursor.execute("""
CREATE TABLE IF NOT EXISTS venues (
    venue_id INT PRIMARY KEY,
    venue_name VARCHAR(255),
    city VARCHAR(100),
    country VARCHAR(100),
    capacity INT
);
""")

# -----------------------------
# 4. series table
# -----------------------------
cursor.execute("""
CREATE TABLE IF NOT EXISTS series (
    series_id INT PRIMARY KEY,
    series_name VARCHAR(255),
    host_country VARCHAR(100),
    match_type VARCHAR(20),
    start_date DATE,
    end_date DATE,
    total_matches INT
);
""")

# -----------------------------
# 5. matches table
# -----------------------------
cursor.execute("""
CREATE TABLE IF NOT EXISTS matches (
    match_id INT PRIMARY KEY,
    series_id INT,
    match_date DATE,
    description VARCHAR(255),
    venue_id INT,
    team1_id INT,
    team2_id INT,
    winner_team_id INT NULL,
    victory_margin INT NULL,
    victory_type VARCHAR(20),
    toss_winner_id INT,
    toss_decision VARCHAR(10),
    CONSTRAINT fk_series FOREIGN KEY (series_id) REFERENCES series(series_id),
    CONSTRAINT fk_venue FOREIGN KEY (venue_id) REFERENCES venues(venue_id),
    CONSTRAINT fk_team1 FOREIGN KEY (team1_id) REFERENCES teams(team_id),
    CONSTRAINT fk_team2 FOREIGN KEY (team2_id) REFERENCES teams(team_id),
    CONSTRAINT fk_winner FOREIGN KEY (winner_team_id) REFERENCES teams(team_id),
    CONSTRAINT fk_toss FOREIGN KEY (toss_winner_id) REFERENCES teams(team_id)
);
""")

# -----------------------------
# 6. player_match_stats table
# -----------------------------
cursor.execute("""
CREATE TABLE IF NOT EXISTS player_match_stats (
    id INT AUTO_INCREMENT PRIMARY KEY,
    match_id INT,
    player_id INT,
    runs_scored INT,
    balls_faced INT,
    fours INT,
    sixes INT,
    strike_rate FLOAT,
    wickets_taken INT,
    overs_bowled FLOAT,
    economy_rate FLOAT,
    catches INT,
    stumpings INT,
    CONSTRAINT fk_match FOREIGN KEY (match_id) REFERENCES matches(match_id),
    CONSTRAINT fk_player FOREIGN KEY (player_id) REFERENCES players(player_id)
);
""")

# -----------------------------
# 7. partnerships table
# -----------------------------
cursor.execute("""
CREATE TABLE IF NOT EXISTS partnerships (
    id INT AUTO_INCREMENT PRIMARY KEY,
    match_id INT,
    innings_no INT,
    batsman1_id INT,
    batsman2_id INT,
    runs INT,
    balls_faced INT,
    CONSTRAINT fk_part_match FOREIGN KEY (match_id) REFERENCES matches(match_id),
    CONSTRAINT fk_batsman1 FOREIGN KEY (batsman1_id) REFERENCES players(player_id),
    CONSTRAINT fk_batsman2 FOREIGN KEY (batsman2_id) REFERENCES players(player_id)
);
""")

# Commit and close
conn.commit()
cursor.close()
conn.close()


In [ ]:
sampple_data = ''
# ---------------------
# Insert Sample Data
# ---------------------
insert_query = ("""
INSERT INTO matches (
    match_id, series_id, match_date, description, venue_id, 
    team1_id, team2_id, winner_team_id, victory_margin, victory_type, 
    toss_winner_id, toss_decision
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
ON DUPLICATE KEY UPDATE
    series_id = VALUES(series_id),
    match_date = VALUES(match_date),
    description = VALUES(description),
    venue_id = VALUES(venue_id),
    team1_id = VALUES(team1_id),
    team2_id = VALUES(team2_id),
    winner_team_id = VALUES(winner_team_id),
    victory_margin = VALUES(victory_margin),
    victory_type = VALUES(victory_type),
    toss_winner_id = VALUES(toss_winner_id),
    toss_decision = VALUES(toss_decision);
"""
,(sampple_data))

In [ ]:
import requests

url = "https://cricbuzz-cricket.p.rapidapi.com/matches/v1/recent"
headers = {
    "x-rapidapi-key": "3eaece787amshfe1adf78b14d67bp1c781fjsnebfd830416ef",
    "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
}

response = requests.get(url, headers=headers)
data = response.json()

match_ids = []

for category in data.get("typeMatches", []):
    for series_wrapper in category.get("seriesMatches", []):
        series = series_wrapper.get("seriesAdWrapper", {})
        matches = series.get("matches", [])
        for match in matches:
            match_id = match.get("matchInfo", {}).get("matchId")
            if match_id:
                match_ids.append(match_id)




[130124, 130140, 130146, 130113, 130129, 130157, 105850, 105858, 116822, 116828, 111985, 130993, 130998, 133495, 133484, 133468, 133479, 119843, 119834, 134986, 135035, 135002, 135024, 135008, 134991, 134975, 135013, 135096, 135090, 135101, 135079, 133699, 133693]
Total matches collected: 33


In [ ]:
import requests
from datetime import datetime
url = "https://cricbuzz-cricket.p.rapidapi.com/mcenter/v1/41881"

headers = {
	"x-rapidapi-key": "3eaece787amshfe1adf78b14d67bp1c781fjsnebfd830416ef",
	"x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
}

response = requests.get(url, headers=headers)

match_data = response.json()
print(match_data)
match_info = match_data.get("matchInfo", {})


match_id = match_info.get("matchId")
series_id = match_info.get("series", {}).get("id")
match_date_ts = match_info.get("matchStartTimestamp")
match_date = datetime.fromtimestamp(match_date_ts / 1000).date() if match_date_ts else None
description = match_info.get("matchDescription")
venue_id = match_info.get("venue", {}).get("id")
team1_id = match_info.get("team1", {}).get("id")
team2_id = match_info.get("team2", {}).get("id")
winner_team_id = match_info.get("result", {}).get("winningteamId")

# Victory margin and type
winning_margin = match_info.get("result", {}).get("winningMargin")
win_by_runs = match_info.get("result", {}).get("winByRuns")
win_by_innings = match_info.get("result", {}).get("winByInnings")

if win_by_runs:
    victory_type = "runs"
elif win_by_innings:
    victory_type = "wickets"
else:
    victory_type = "NR"

# Toss info
toss_winner_id = match_info.get("tossResults", {}).get("tossWinnerId")
toss_decision = match_info.get("tossResults", {}).get("decision")
if toss_decision:
    toss_decision = toss_decision.lower()  # standardize as 'bat'/'bowl'

# Print to check
print("match_id:", match_id)
print("series_id:", series_id)
print("match_date:", match_date)
print("description:", description)
print("venue_id:", venue_id)
print("team1_id:", team1_id)
print("team2_id:", team2_id)
print("winner_team_id:", winner_team_id)
print("winning_margin:", winning_margin)
print("victory_type:", victory_type)
print("toss_winner_id:", toss_winner_id)
print("toss_decision:", toss_decision)


{'matchid': 41881, 'seriesid': 3926, 'seriesname': 'ACC U19 Asia Cup 2021', 'matchdesc': 'Group A', 'matchformat': 'ODI', 'startdate': 1640410200000, 'enddate': 1640439000000, 'state': 'Complete', 'status': 'Afghanistan U19 won by 140 runs', 'team1': {'teamid': 223, 'teamname': 'Afghanistan U19', 'teamsname': 'AFGU19', 'isfullmember': False, 'isassociated': False, 'isleagueteam': False, 'iswomenteam': False, 'isheader': False, 'isactive': False, 'teampriority': '', 'isvideopresent': False, 'imageid': 0, 'countryname': '', 'belongsto': ''}, 'team2': {'teamid': 283, 'teamname': 'United Arab Emirates U19', 'teamsname': 'UAEU19', 'isfullmember': False, 'isassociated': False, 'isleagueteam': False, 'iswomenteam': False, 'isheader': False, 'isactive': False, 'teampriority': '', 'isvideopresent': False, 'imageid': 0, 'countryname': '', 'belongsto': ''}, 'umpire1': {'id': 7540, 'name': 'Naresh Shah', 'country': 'KEN'}, 'umpire2': {'id': 7244, 'name': 'Sarika Prasad', 'country': 'SIN'}, 'umpire

In [ ]:
import json
import requests
from datetime import datetime
def fetch_latest_matches():
    url = "https://cricbuzz-cricket.p.rapidapi.com/matches/v1/recent"
    headers = {
        "X-RapidAPI-Key": "3eaece787amshfe1adf78b14d67bp1c781fjsnebfd830416ef",
        "X-RapidAPI-Host": "cricbuzz-cricket.p.rapidapi.com"
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    return []
data = fetch_latest_matches()
matches_table = []
match_ids = []
series_ids =[]
def toss_decisions(id):
    url = f"https://cricbuzz-cricket.p.rapidapi.com/mcenter/v1/{id}"

    headers = {
        "x-rapidapi-key": "3eaece787amshfe1adf78b14d67bp1c781fjsnebfd830416ef",
        "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers)

    data = response.json()
    # Get teams info
    team1_id = data.get("team1", {}).get("teamid")
    team1_name = data.get("team1", {}).get("teamname")
    team2_id = data.get("team2", {}).get("teamid")
    team2_name = data.get("team2", {}).get("teamname")
    toss_status = data.get("tossstatus", "")

    # Parse tossstatus
    toss_winner_id = None
    toss_decision = None

    if toss_status is not None:
        parts = toss_status.split(" opt to ")
        if len(parts) == 2:
            toss_winner_name = parts[0].strip()      # "Bangladesh"
            toss_decision = parts[1].strip().lower() # "bat" or "bowl"

            # Map to team_id
            if toss_winner_name == team1_name:
                toss_winner_id = team1_id
            elif toss_winner_name == team2_name:
                toss_winner_id = team2_id
    return toss_winner_id,toss_decision




for category in data.get("typeMatches", []):
    for series in category.get("seriesMatches", []):
        series_wrapper = series.get("seriesAdWrapper", {})
        for match in series_wrapper.get("matches", []):
            info = match.get("matchInfo", {})
            match_id = info.get("matchId")
            series_id = info.get("seriesId")
            match_date =  datetime.fromtimestamp(int(info.get("startDate"))/1000).strftime("%Y-%m-%d %H:%M:%S") # timestamp
            description = info.get("matchDesc")
            venue_id = info.get("venueInfo", {}).get("id")
            team1_id = info.get("team1", {}).get("teamId")
            team2_id = info.get("team2", {}).get("teamId")
            
            # Winner info
            winner_team_id = None
            victory_margin = None
            victory_type = "NR"
            status = info.get("status", "")
            if "won by" in status:
                parts = status.split(" won by ")
                winner_team_name = parts[0].strip()
                winner_team_id = team1_id if winner_team_name == info.get("team1", {}).get("teamName") else team2_id
                margin_parts = parts[1].split(" ")
                victory_margin = int(margin_parts[0])
                victory_type = margin_parts[1] if margin_parts[1].lower() in ["runs", "wickets"] else "NR"
            
            toss_winner_id ,toss_decision = toss_decisions(match_id)
            
            match_ids.append(match_id)
            series_ids.append(series_id)
            # cursor.execute("""
            #     INSERT INTO matches 
            #     (match_id, series_id, match_date, description, venue_id, team1_id, team2_id, 
            #     winner_team_id, victory_margin, victory_type, toss_winner_id, toss_decision)
            #     VALUES (%(match_id)s, %(series_id)s, %(match_date)s, %(description)s, %(venue_id)s, %(team1_id)s, %(team2_id)s,
            #             %(winner_team_id)s, %(victory_margin)s, %(victory_type)s, %(toss_winner_id)s, %(toss_decision)s)
            #     ON DUPLICATE KEY UPDATE 
            #         series_id=VALUES(series_id), match_date=VALUES(match_date), description=VALUES(description),
            #         venue_id=VALUES(venue_id), team1_id=VALUES(team1_id), team2_id=VALUES(team2_id),
            #         winner_team_id=VALUES(winner_team_id), victory_margin=VALUES(victory_margin),
            #         victory_type=VALUES(victory_type), toss_winner_id=VALUES(toss_winner_id),
            #         toss_decision=VALUES(toss_decision)
            # """, {
            #     "match_id": match_id,
            #     "series_id": series_id,
            #     "match_date": match_date,
            #     "description": description,
            #     "venue_id": venue_id,
            #     "team1_id": team1_id,
            #     "team2_id": team2_id,
            #     "winner_team_id": winner_team_id,
            #     "victory_margin": victory_margin,
            #     "victory_type": victory_type,
            #     "toss_winner_id": toss_winner_id,
            #     "toss_decision": toss_decision
            # })


            # conn.commit()
            # print("done",description)



In [ ]:
import mysql.connector

# 1️⃣ Connect to MySQL
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="2741",
    database="cricketdb"
)
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS matches (
    match_id INT PRIMARY KEY,
    series_id INT,
    match_date DATE,
    description VARCHAR(255),
    venue_id INT,
    team1_id INT,
    team2_id INT,
    winner_team_id INT NULL,
    victory_margin INT NULL,
    victory_type VARCHAR(20),
    toss_winner_id INT,
    toss_decision VARCHAR(10),
    CONSTRAINT fk_series FOREIGN KEY (series_id) REFERENCES series(series_id),
    CONSTRAINT fk_venue FOREIGN KEY (venue_id) REFERENCES venues(venue_id),
    CONSTRAINT fk_team1 FOREIGN KEY (team1_id) REFERENCES teams(team_id),
    CONSTRAINT fk_team2 FOREIGN KEY (team2_id) REFERENCES teams(team_id),
    CONSTRAINT fk_winner FOREIGN KEY (winner_team_id) REFERENCES teams(team_id),
    CONSTRAINT fk_toss FOREIGN KEY (toss_winner_id) REFERENCES teams(team_id)
);
""")

cursor.execute(create_table_query)



In [ ]:
for serie_id in series_ids:
    if not series_id:  # skip if series_id is None or 0
        continue

    url = f"https://cricbuzz-cricket.p.rapidapi.com/series/v1/{serie_id}"
    headers = {
        "x-rapidapi-key": "3eaece787amshfe1adf78b14d67bp1c781fjsnebfd830416ef",
        "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
    }
    response = requests.get(url, headers=headers, timeout=10)
    series_info = response.json()
    print(series_info)
    host_url = f"https://cricbuzz-cricket.p.rapidapi.com/series/v1/{serie_id}/venues"
    host_response = requests.get(host_url, headers=headers)
    host_res = host_response.json()
    # print(series_info)

    # Get seriesVenue safely
    series_venues = host_res.get("seriesVenue", [])

    if series_venues:  # make sure it's not empty
        host_country = series_venues[0].get("country")
    else:
        host_country = None   

    for item in series_info.get("matchDetails", []):
        match_map = item.get("matchDetailsMap")
        if not match_map:  # Skip ads or other non-match entries
            continue
    
        for match in match_map.get("match", []):
            info = match.get("matchInfo", {})
            match_id = info.get("matchId")
            series_name = info.get("seriesName")
            match_desc = info.get("matchDesc", "")
            match_type = info.get("matchFormat", "")
            start_date = datetime.fromtimestamp(int(info.get("startDate"))/1000).strftime("%Y-%m-%d %H:%M:%S") if info.get("startDate") else None
            end_date = datetime.fromtimestamp(int(info.get("endDate"))/1000).strftime("%Y-%m-%d %H:%M:%S") if info.get("endDate") else None
            state = info.get("state", "")
            status = info.get("status", "")
            total_matches = sum(
            1 for item in series_info.get("matchDetails", [])
            if item.get("matchDetailsMap") and item["matchDetailsMap"].get("match"))

            # Generate INSERT statement
            insert_sql = """
                INSERT INTO series 
                (series_id, series_name, host_country, match_type, start_date, end_date, total_matches)
                VALUES (%s, %s, %s, %s, %s, %s, %s)
                ON DUPLICATE KEY UPDATE 
                series_name=%s, host_country=%s, match_type=%s, start_date=%s, end_date=%s, total_matches=%s
                """
            cursor.execute(insert_sql, (
                serie_id, series_name, host_country, match_type, start_date, end_date, total_matches,
                series_name, host_country, match_type, start_date, end_date, total_matches  # repeat for update
            ))

            conn.commit()
            print(f"Insert sql")



{'matchDetails': [{'matchDetailsMap': {'key': 'Tue, 09 Sep 2025', 'match': [{'matchInfo': {'matchId': 130008, 'seriesId': 10587, 'seriesName': 'Asia Cup 2025', 'matchDesc': '1st Match, Group B', 'matchFormat': 'T20', 'startDate': '1757428200000', 'endDate': '1757440800000', 'state': 'Complete', 'status': 'Afghanistan won by 94 runs', 'team1': {'teamId': 96, 'teamName': 'Afghanistan', 'teamSName': 'AFG', 'imageId': 172188}, 'team2': {'teamId': 8, 'teamName': 'Hong Kong', 'teamSName': 'HK', 'imageId': 172122}, 'venueInfo': {'id': 94, 'ground': 'Sheikh Zayed Stadium', 'city': 'Abu Dhabi', 'timezone': '+04:00'}, 'currBatTeamId': 96, 'seriesStartDt': '1757376000000', 'seriesEndDt': '1759190400000', 'isTimeAnnounced': True}, 'matchScore': {'team1Score': {'inngs1': {'inningsId': 1, 'runs': 188, 'wickets': 6, 'overs': 20.0}}, 'team2Score': {'inngs1': {'inningsId': 2, 'runs': 94, 'wickets': 9, 'overs': 20.0}}}}], 'seriesId': 10587}}, {'adDetail': {'name': 'native_news_index_random_1', 'layout':

In [8]:
for venue_id in venue_ids:
    if not venue_id:  # skip if venue_id is None or 0
        continue

    url = f"https://cricbuzz-cricket.p.rapidapi.com/venues/v1/{venue_id}"
    headers = {
        "x-rapidapi-key": "3eaece787amshfe1adf78b14d67bp1c781fjsnebfd830416ef",
        "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
    }

    try:
        response = requests.get(url, headers=headers, timeout=10)
    except requests.exceptions.RequestException as e:
        print(f"Request failed for venue {venue_id}: {e}")
        continue

    if response.status_code != 200 or not response.text.strip():
        print(f"Bad response for venue {venue_id}, status code: {response.status_code}")
        continue

    try:
        venue_info = response.json()
    except json.JSONDecodeError:
        print(f"Invalid JSON for venue {venue_id}")
        continue

    # Extract info with defaults
    ground_name = venue_info.get("ground", "Unknown")
    city = venue_info.get("city", "")
    country = venue_info.get("country", "")

    # Insert into DB (ignore duplicates)
    cursor.execute("""
        INSERT IGNORE INTO venues (venue_id, venue_name, city, country)
        VALUES (%s, %s, %s, %s)
    """, (venue_id, ground_name, city, country))
    conn.commit()

    print(f"Inserted venue: {ground_name}, {city}, {country}")


Inserted venue: Dubai International Cricket Stadium, Dubai, United Arab Emirates
Inserted venue: Dubai International Cricket Stadium, Dubai, United Arab Emirates
Inserted venue: Dubai International Cricket Stadium, Dubai, United Arab Emirates
Inserted venue: Sheikh Zayed Stadium, Abu Dhabi, United Arab Emirates
Inserted venue: Dubai International Cricket Stadium, Dubai, United Arab Emirates
Inserted venue: Sheikh Zayed Stadium, Abu Dhabi, United Arab Emirates
Inserted venue: The Village, Dublin, Ireland
Inserted venue: The Village, Dublin, Ireland
Inserted venue: Providence Stadium, Guyana, West Indies
Inserted venue: Providence Stadium, Guyana, West Indies
Inserted venue: Trent Bridge, Nottingham, England
Inserted venue: Ian Healy Oval, Brisbane, Australia
Inserted venue: Ian Healy Oval, Brisbane, Australia
Inserted venue: Karen Rolton Oval, Adelaide, South Australia, Australia
Inserted venue: Allan Border Field, Brisbane, Australia
Bad response for venue 1727, status code: 500
Insert

In [ ]:
import requests
for id in match_ids:
    url = f"https://cricbuzz-cricket.p.rapidapi.com/mcenter/v1/{id}"

    headers = {
        "x-rapidapi-key": "3eaece787amshfe1adf78b14d67bp1c781fjsnebfd830416ef",
        "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers)

    data = response.json()
    # Get teams info
    team1_id = data.get("team1", {}).get("teamid")
    team1_name = data.get("team1", {}).get("teamname")
    team2_id = data.get("team2", {}).get("teamid")
    team2_name = data.get("team2", {}).get("teamname")
    toss_status = data.get("tossstatus", "")

    # Parse tossstatus
    toss_winner_id = None
    toss_decision = None

    if toss_status is not None:
        parts = toss_status.split(" opt to ")
        if len(parts) == 2:
            toss_winner_name = parts[0].strip()      # "Bangladesh"
            toss_decision = parts[1].strip().lower() # "bat" or "bowl"

            # Map to team_id
            if toss_winner_name == team1_name:
                toss_winner_id = team1_id
            elif toss_winner_name == team2_name:
                toss_winner_id = team2_id

    print("Toss Winner ID:", toss_winner_id)
    print("Toss Decision:", toss_decision)


Toss Winner ID: 6
Toss Decision: bowl
Toss Winner ID: 6
Toss Decision: bowl
Toss Winner ID: 2
Toss Decision: bowl
Toss Winner ID: 3
Toss Decision: bowl
Toss Winner ID: 2
Toss Decision: bat
Toss Winner ID: 6
Toss Decision: bowl
Toss Winner ID: None
Toss Decision: None
Toss Winner ID: 9
Toss Decision: bowl


: 

In [3]:
import mysql.connector

# 1️⃣ Connect to MySQL
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="2741",
    database="cricketdb"
)
cursor = conn.cursor()

# 2️⃣ Create table if it doesn't exist
create_table_query = """CREATE TABLE IF NOT EXISTS partnerships (
    id INT AUTO_INCREMENT PRIMARY KEY,
    match_id INT,
    innings_no INT,
    batsman1_id INT,
    batsman2_id INT,
    runs INT,
    balls_faced INT,
    FOREIGN KEY (match_id) REFERENCES matches(match_id),
    FOREIGN KEY (batsman1_id) REFERENCES players(player_id),
    FOREIGN KEY (batsman2_id) REFERENCES players(player_id)
);

"""

cursor.execute(create_table_query)



In [5]:
import json
import requests

for match_id in match_ids:
    url = f"https://cricbuzz-cricket.p.rapidapi.com/mcenter/v1/{match_id}/hscard"
    headers = {
        "x-rapidapi-key": "3eaece787amshfe1adf78b14d67bp1c781fjsnebfd830416ef",
        "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers)
    data = response.json()

    # Pre-fetch all player IDs to avoid repeated DB queries
    cursor.execute("SELECT player_id FROM players")
    valid_players = set(row for row in cursor.fetchall())

    # Check if match exists once
    cursor.execute("SELECT match_id FROM matches WHERE match_id = %s", (match_id,))
    match_exists_flag = cursor.fetchone() is not None

    for innings in data.get('scorecard', []):
        innings_no = innings.get('inningsid')
        partnerships = innings.get('partnership', {}).get('partnership', [])

        for p in partnerships:
            bat1 = p.get('bat1id')
            bat2 = p.get('bat2id')

            # Skip if match doesn't exist or players don't exist
            if not match_exists_flag or bat1 not in valid_players or bat2 not in valid_players:
                continue

            cursor.execute("""
                INSERT INTO partnerships (match_id, innings_no, batsman1_id, batsman2_id, runs, balls_faced)
                VALUES (%s, %s, %s, %s, %s, %s)
            """, (match_id, innings_no, bat1, bat2, p.get('totalruns', 0), p.get('totalballs', 0)))

    conn.commit()  # Commit after all partnerships for this match


In [1]:
import mysql.connector

# 1️⃣ Connect to MySQL
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="2741",
    database="cricketdb"
)
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS matches (
    match_id INT PRIMARY KEY,
    series_id INT,
    match_date DATE,
    description VARCHAR(255),
    venue_id INT,
    team1_id INT,
    team2_id INT,
    winner_team_id INT NULL,
    victory_margin INT NULL,
    victory_type VARCHAR(20),
    toss_winner_id INT,
    toss_decision VARCHAR(10),
    CONSTRAINT fk_series FOREIGN KEY (series_id) REFERENCES series(series_id),
    CONSTRAINT fk_venue FOREIGN KEY (venue_id) REFERENCES venues(venue_id),
    CONSTRAINT fk_team1 FOREIGN KEY (team1_id) REFERENCES teams(team_id),
    CONSTRAINT fk_team2 FOREIGN KEY (team2_id) REFERENCES teams(team_id),
    CONSTRAINT fk_winner FOREIGN KEY (winner_team_id) REFERENCES teams(team_id),
    CONSTRAINT fk_toss FOREIGN KEY (toss_winner_id) REFERENCES teams(team_id)
);
""")





In [8]:
import requests
import mysql.connector
import time

def update_venues_tables(venue_ids, conn):
    cursor = conn.cursor()

    for venue_id in venue_ids:
        # Skip invalid IDs
        if not venue_id or not isinstance(venue_id, int):
            print(f"Skipping invalid venue_id: {venue_id}")
            continue

        # API request setup
        url = f"https://cricbuzz-cricket.p.rapidapi.com/venues/v1/{venue_id}"
        headers = {
            "x-rapidapi-key": "3eaece787amshfe1adf78b14d67bp1c781fjsnebfd830416ef",
            "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
        }

        # Fetch venue data
        try:
            response = requests.get(url, headers=headers, timeout=10)
        except requests.exceptions.RequestException as e:
            print(f"Request failed for venue {venue_id}: {e}")
            continue

        if response.status_code != 200 or not response.text.strip():
            print(f"Bad response for venue {venue_id}, status code: {response.status_code}")
            continue

        try:
            venue_info = response.json()
        except Exception as e:
            print(f"Invalid JSON for venue {venue_id}: {e}")
            continue

        # Extract fields
        venue_name = venue_info.get("ground", "Unknown")
        city = venue_info.get("city", "")
        country = venue_info.get("country", "")

        # Capacity handling
        capacity = None
        capacity_raw = venue_info.get("capacity") or venue_info.get("profile", {}).get("capacity")

        if capacity_raw:
            try:
                if isinstance(capacity_raw, int):
                    capacity = capacity_raw
                elif isinstance(capacity_raw, str):
                    digits = "".join(c for c in capacity_raw if c.isdigit())
                    capacity = int(digits) if digits else None
            except Exception as e:
                print(f"Capacity parsing error for {venue_id}: {capacity_raw} -> {e}")
                capacity = None

        print(f"Venue {venue_id} raw capacity: {capacity_raw} | Parsed: {capacity}")

        # Insert or Update into DB
        try:
            cursor.execute("""
                INSERT INTO venues (venue_id, venue_name, city, country, capacity)
                VALUES (%s, %s, %s, %s, %s)
                ON DUPLICATE KEY UPDATE
                    venue_name = VALUES(venue_name),
                    city = VALUES(city),
                    country = VALUES(country),
                    capacity = VALUES(capacity)
            """, (venue_id, venue_name, city, country, capacity))
            conn.commit()
            print(f"Upserted venue: {venue_name}, {city}, {country}, {capacity}")
        except mysql.connector.Error as e:
            print(f"MySQL error for venue {venue_id}: {e}")
            continue

        # Avoid hitting API rate limits
        time.sleep(0.5)


In [11]:
def update_series_table(series_ids, conn):
    import requests
    from datetime import datetime

    cursor = conn.cursor()
    headers = {
        "x-rapidapi-key": "3eaece787amshfe1adf78b14d67bp1c781fjsnebfd830416ef",
        "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
    }

    for serie_id in series_ids:
        if not serie_id:
            continue

        try:
            # 1️⃣ Fetch main series info
            url = f"https://cricbuzz-cricket.p.rapidapi.com/series/v1/{serie_id}"
            response = requests.get(url, headers=headers, timeout=10)
            if response.status_code != 200:
                print(f"Failed to fetch series {serie_id}")
                continue

            series_info = response.json()
            series_name = series_info.get("seriesName", "").strip()
            match_type = series_info.get("matchFormat", "").strip()

            start_date = (
                datetime.fromtimestamp(int(series_info.get("startDate", 0)) / 1000)
                .strftime("%Y-%m-%d %H:%M:%S") 
                if series_info.get("startDate") else None
            )
            end_date = (
                datetime.fromtimestamp(int(series_info.get("endDate", 0)) / 1000)
                .strftime("%Y-%m-%d %H:%M:%S") 
                if series_info.get("endDate") else None
            )

            total_matches = sum(
                1 for item in series_info.get("matchDetails", [])
                if item.get("matchDetailsMap") and item["matchDetailsMap"].get("match")
            )

            # 2️⃣ Fetch series venues (host country)
            host_url = f"https://cricbuzz-cricket.p.rapidapi.com/series/v1/{serie_id}/venues"
            host_response = requests.get(host_url, headers=headers, timeout=10)
            host_res = host_response.json()
            series_venues = host_res.get("seriesVenue", [])
            host_country = series_venues[0].get("country") if series_venues else None

            # 3️⃣ Insert or update in DB
            insert_sql = """
                INSERT INTO series 
                (series_id, series_name, host_country, match_type, start_date, end_date, total_matches)
                VALUES (%s, %s, %s, %s, %s, %s, %s)
                ON DUPLICATE KEY UPDATE 
                    series_name=VALUES(series_name),
                    host_country=VALUES(host_country),
                    match_type=VALUES(match_type),
                    start_date=VALUES(start_date),
                    end_date=VALUES(end_date),
                    total_matches=VALUES(total_matches)
            """
            cursor.execute(insert_sql, (
                serie_id, series_name, host_country, match_type, start_date, end_date, total_matches
            ))
            conn.commit()
            print(f"Updated series: {series_name}")

        except Exception as e:
            print(f"Error updating series {serie_id}: {e}")

    cursor.close()


In [10]:
import json
import requests
from datetime import datetime

def update_match_table():
    import mysql.connector

    # -----------------------
    # Database connection
    # -----------------------
    conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="2741",
        database="cricketdb"
    )
    cursor = conn.cursor()
    # -----------------------
    # Fetch latest matches
    # -----------------------
    def fetch_latest_matches():
        url = "https://cricbuzz-cricket.p.rapidapi.com/matches/v1/recent"
        headers = {
            "X-RapidAPI-Key": "3eaece787amshfe1adf78b14d67bp1c781fjsnebfd830416ef",
            "X-RapidAPI-Host": "cricbuzz-cricket.p.rapidapi.com"
        }
        response = requests.get(url, headers=headers)
        return response.json() if response.status_code == 200 else {}
    def batch_insert(cursor, query, data_list, batch_size=20):
        for i in range(0, len(data_list), batch_size):
            batch = data_list[i:i+batch_size]
            cursor.executemany(query, batch)
            conn.commit() 
    # -----------------------
    # Fetch toss info
    # -----------------------
    def toss_decisions(match_id):
        url = f"https://cricbuzz-cricket.p.rapidapi.com/mcenter/v1/{match_id}"
        headers = {
            "x-rapidapi-key": "3eaece787amshfe1adf78b14d67bp1c781fjsnebfd830416ef",
            "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
        }
        response = requests.get(url, headers=headers)
        data = response.json()

        team1 = data.get("team1", {})
        team2 = data.get("team2", {})
        toss_status = data.get("tossstatus", "")

        toss_winner_id = None
        toss_decision = None

        if toss_status:
            parts = toss_status.split(" opt to ")
            if len(parts) == 2:
                toss_winner_name = parts[0].strip()
                toss_decision = parts[1].strip().lower()
                if toss_winner_name == team1.get("teamname"):
                    toss_winner_id = team1.get("teamid")
                elif toss_winner_name == team2.get("teamname"):
                    toss_winner_id = team2.get("teamid")

        return toss_winner_id, toss_decision

    # -----------------------
    # Main insertion loop
    # -----------------------
    data = fetch_latest_matches()
    series_ids=[]
    venues_ids =[]

    for category in data.get("typeMatches", []):
        for series in category.get("seriesMatches", []):
            series_wrapper = series.get("seriesAdWrapper", {})
            series_id = series_wrapper.get("seriesId")
            series_name = series_wrapper.get("seriesName")
            series_ids.append(series_id)
            # print(series_id)
            for match in series_wrapper.get("matches", []):
                info = match.get("matchInfo", {})
                match_id = info.get("matchId")
                match_date_ts = info.get("startDate")
                match_date = datetime.fromtimestamp(int(match_date_ts)/1000).strftime("%Y-%m-%d %H:%M:%S") if match_date_ts else None
                description = info.get("matchDesc")
                venue_id = info.get("venueInfo", {}).get("id")
                venues_ids.append(venue_id)
                venue_name = info.get("venueInfo", {}).get("ground")

                team1 = info.get("team1", {})
            
                team2= info.get("team2", {})
            

                # Winner info
                winner_team_id = None
                victory_margin = None
                victory_type = "NR"
                team1_id = team1.get("teamId")
                team2_id = team2.get("teamId")

                status = info.get("status", "")
                if "won by" in status:
                    parts = status.split(" won by ")
                    winner_team_name = parts[0].strip()
                    winner_team_id = team1_id if winner_team_name == info.get("team1", {}).get("teamName") else team2_id
                    margin_parts = parts[1].split(" ")
                    try:
                        victory_margin = int(margin_parts[0])
                        victory_type = margin_parts[1] if margin_parts[1].lower() in ["runs", "wickets"] else "NR"
                    except:
                        victory_margin = None

                # Toss info
                toss_winner_id, toss_decision = toss_decisions(match_id)
                match_values = []
                series_values = []
                venue_values = []
                
                # Inside loop, collect
                series_values.append((series_id, series_name))
                venue_values.append((venue_id,venue_name))
                match_values.append((
                    match_id, series_id, match_date, description, venue_id, team1_id, team2_id,
                    winner_team_id, victory_margin, victory_type, toss_winner_id, toss_decision
                ))
                teams_values = []

                for team in [team1, team2]:
                    # team is a dict from the API
                    team_id = team.get("teamId")
                    team_name = team.get("teamName")
                    if team_id and team_name:
                        # Must be a tuple (team_id, team_name), not a dict
                        teams_values.append((team_id, team_name))

                batch_insert(cursor, """
                    INSERT IGNORE INTO teams (team_id, team_name)
                    VALUES (%s, %s)
                """, teams_values)

                # Insert match
                            # Insert match
            

                batch_insert(cursor,"""
                    INSERT IGNORE INTO venues (venue_id, venue_name)
                VALUES (%s, %s)
                    """,venue_values)

                # batch_insert(cursor, """
                #     INSERT INTO matches
                #     (match_id, series_id, match_date, description, venue_id, team1_id, team2_id,
                #     winner_team_id, victory_margin, victory_type, toss_winner_id, toss_decision)
                #     VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                #     ON DUPLICATE KEY UPDATE
                #         series_id=VALUES(series_id), match_date=VALUES(match_date), description=VALUES(description),
                #         venue_id=VALUES(venue_id), team1_id=VALUES(team1_id), team2_id=VALUES(team2_id),
                #         winner_team_id=VALUES(winner_team_id), victory_margin=VALUES(victory_margin),
                #         victory_type=VALUES(victory_type), toss_winner_id=VALUES(toss_winner_id),
                #         toss_decision=VALUES(toss_decision)
                # """, match_values)
                # conn.commit()
                # print("done",description)


    print("All matches processed successfully.")
    update_series_table(series_ids,conn)
    #update_venues_tables(venues_ids,conn)




In [12]:
update_match_table()

All matches processed successfully.
Updated series: 
Updated series: 
Updated series: 
Updated series: 
Updated series: 
Updated series: 
Updated series: 
Updated series: 
Updated series: 
Updated series: 
Updated series: 
Updated series: 
Updated series: 
Updated series: 
Updated series: 


In [ ]:
import requests
import mysql.connector
from datetime import datetime

# -------------------
# Database connection
# -------------------
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="2741",
    database="cricketdb"
)
cursor = conn.cursor()

# -------------------
# Create series table if not exists
# -------------------
create_table_query = """
CREATE TABLE IF NOT EXISTS series (
    series_id INT PRIMARY KEY,
    series_name VARCHAR(255),
    host_country VARCHAR(100),
    start_date DATE,
    end_date DATE,
    total_matches INT
)
"""
cursor.execute(create_table_query)
conn.commit()

# -------------------
# RapidAPI config
# -------------------
API_KEY = "3eaece787amshfe1adf78b14d67bp1c781fjsnebfd830416ef"
HEADERS = {
    "x-rapidapi-key": API_KEY,
    "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
}

# -------------------
# Fetch series list
# -------------------
series_url = "https://cricbuzz-cricket.p.rapidapi.com/series/v1/archives/international"
try:
    series_response = requests.get(series_url, headers=HEADERS, timeout=10)
    series_response.raise_for_status()
    series_data = series_response.json()
except requests.RequestException as e:
    print(f"Error fetching series list: {e}")
    series_data = {}
except ValueError as e:  # JSON decode error
    print(f"Error parsing series JSON: {e}")
    series_data = {}

# -------------------
# Helper: get total matches for a series
# -------------------
def get_total_matches(series_id):
    url = f"https://cricbuzz-cricket.p.rapidapi.com/series/v1/{series_id}"
    try:
        response = requests.get(url, headers=HEADERS, timeout=10)
        response.raise_for_status()
        data = response.json()
    except requests.RequestException as e:
        print(f"Error fetching matches for series {series_id}: {e}")
        return 0
    except ValueError as e:
        print(f"Error parsing matches JSON for series {series_id}: {e}")
        return 0

    total_matches = 0
    for item in data.get("matchDetails", []):
        match_map = item.get("matchDetailsMap")
        if match_map and "match" in match_map:
            total_matches += len(match_map["match"])
    return total_matches

# -------------------
# Insert series data
# -------------------
for year_block in series_data.get("seriesMapProto", []):
    for s in year_block.get("series", []):
        series_id = s["id"]
        series_name = s["name"]
        start_date = datetime.fromtimestamp(int(s["startDt"]) / 1000).strftime("%Y-%m-%d")
        end_date = datetime.fromtimestamp(int(s["endDt"]) / 1000).strftime("%Y-%m-%d")

        # Fetch host country from venues
        venue_url = f"https://cricbuzz-cricket.p.rapidapi.com/series/v1/{series_id}/venues"
        try:
            venue_response = requests.get(venue_url, headers=HEADERS, timeout=10)
            venue_response.raise_for_status()
            venue_data = venue_response.json()
            venues = venue_data.get("seriesVenue", [])
            host_country = venues[0].get("country") if venues else None
        except requests.RequestException as e:
            print(f"Warning: Could not fetch venues for series {series_id}: {e}")
            host_country = None
        except ValueError as e:
            print(f"Warning: Could not parse venue JSON for series {series_id}: {e}")
            host_country = None

        # Get total matches from matches API
        total_matches = get_total_matches(series_id)

        # Insert into DB
        insert_query = """
        INSERT INTO series (series_id, series_name, host_country, start_date, end_date, total_matches)
        VALUES (%s, %s, %s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE
            series_name = VALUES(series_name),
            host_country = VALUES(host_country),
            start_date = VALUES(start_date),
            end_date = VALUES(end_date),
            total_matches = VALUES(total_matches)
        """
        cursor.execute(insert_query, (series_id, series_name, host_country, start_date, end_date, total_matches))
        conn.commit()
        print(f"Inserted/Updated series {series_name} with {total_matches} matches.")

cursor.close()
conn.close()
print("Series table updated successfully!")


Inserted/Updated series ICC Mens T20 World Cup Africa Regional Final 2025 with 20 matches.
Inserted/Updated series England tour of Ireland, 2025 with 3 matches.
Inserted/Updated series Namibia tour of Zimbabwe 2025 with 3 matches.
Inserted/Updated series Mozambique tour of Eswatini 2025 with 5 matches.
Inserted/Updated series Asia Cup 2025 with 19 matches.
Inserted/Updated series Sweden tour of Isle Of Man 2025 with 4 matches.
Inserted/Updated series South Africa tour of England, 2025 with 6 matches.
Inserted/Updated series Switzerland tour of Guernsey, 2025 with 3 matches.
Inserted/Updated series Netherlands tour of Bangladesh 2025 with 3 matches.
Inserted/Updated series Eastern Europe Cup 2025 with 7 matches.
Inserted/Updated series United Arab Emirates T20I Tri-Series 2025 with 7 matches.
Inserted/Updated series Sri Lanka tour of Zimbabwe, 2025 with 5 matches.
Inserted/Updated series Belgium tour of Austria 2025 with 4 matches.
Inserted/Updated series Czech Republic tour of Romania 

In [ ]:
import requests



Total matches in series: 19


In [ ]:
def top_odi():
    import json
    import requests
    import mysql.connector
    import mysql.connector
    import requests

    url = "https://cricbuzz-cricket.p.rapidapi.com/stats/v1/topstats/0"

    querystring = {"statsType":"mostRuns","matchType":"2"}

    headers = {
        "x-rapidapi-key": "3eaece787amshfe1adf78b14d67bp1c781fjsnebfd830416ef",
        "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)
    player_ids =[]
    data = response.json()
    for player in data['values']:
        values = player['values']
        player_id = values[0]
        name = values[1]
        matches = values[2]
        innings = values[3]
        runs = values[4]
        avg = values[5]
        
        player_ids.append( player_id,
        )
    # -------------------------------
    # MySQL Connection
    # -------------------------------
    conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="2741",
        database="cricketdb"
    )
    cursor = conn.cursor()

    # -------------------------------
    # Step 1: Create Table
    # -------------------------------
    create_table_query = """
    CREATE TABLE IF NOT EXISTS top_odi_run_scorers (
        player_id INT PRIMARY KEY,
        player_name VARCHAR(100) NOT NULL,
        matches INT,
        innings INT,
        runs INT NOT NULL,
        batting_average DECIMAL(5,2),
        centuries INT
    );
    """
    cursor.execute(create_table_query)
    conn.commit()

    for player in player_ids:
        url = f"https://cricbuzz-cricket.p.rapidapi.com/stats/v1/player/{player}/batting"

        headers = {
            "x-rapidapi-key": "3eaece787amshfe1adf78b14d67bp1c781fjsnebfd830416ef",
            "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
        }

        response = requests.get(url, headers=headers)
        data = response.json()

        player_name = data['appIndex']['seoTitle'].split(" Profile")[0]
        values = data['values']

        # Initialize variables
        matches = innings = runs = avg = hundreds = None

        for row in values:
            stat_name = row['values'][0]
            if stat_name == "Matches":
                matches = row['values'][2]  # ODI column
            elif stat_name == "Innings":
                innings = row['values'][2]
            elif stat_name == "Runs":
                runs = row['values'][2]
            elif stat_name == "Average":
                avg = row['values'][2]
            elif stat_name == "100s":
                hundreds = row['values'][2]

        # Prepare tuple for insertion
        player_tuple = (player, player_name, matches, innings, runs, avg, hundreds)

        # SQL Insert
        insert_query = """
        INSERT INTO top_odi_run_scorers
        (player_id, player_name, matches, innings, runs, batting_average, centuries)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE
        player_name=VALUES(player_name),
        matches=VALUES(matches),
        innings=VALUES(innings),
        runs=VALUES(runs),
        batting_average=VALUES(batting_average),
        centuries=VALUES(centuries);
        """

        cursor.execute(insert_query, player_tuple)
        conn.commit()

    cursor.close()
    conn.close()


In [ ]:
def player_batting_stats():
    import requests
    import mysql.connector
    url = "https://cricbuzz-cricket.p.rapidapi.com/stats/v1/topstats/0"

    querystring = {"statsType":"mostRuns","matchType":"2"}

    headers = {
            "x-rapidapi-key": "3eaece787amshfe1adf78b14d67bp1c781fjsnebfd830416ef",
            "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
        }

    response = requests.get(url, headers=headers, params=querystring)
    player_ids =[]
    data = response.json()
    for player in data['values']:
            values = player['values']
            player_id = values[0]
            name = values[1]
            matches = values[2]
            innings = values[3]
            runs = values[4]
            avg = values[5]
            
            player_ids.append( player_id,
            )
    # MySQL connection
    conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="2741",
        database="cricketdb"
    )
    cursor = conn.cursor()

    # Make sure table includes 'format' column
    create_table_query = """
    CREATE TABLE IF NOT EXISTS player_batting_stats (
        player_id INT,
        player_name VARCHAR(100),
        format VARCHAR(10),
        matches INT,
        innings INT,
        runs INT,
        batting_average DECIMAL(5,2),
        centuries INT,
        PRIMARY KEY(player_id, format)
    );
    """
    cursor.execute(create_table_query)
    conn.commit()

    # Loop through player IDs
    for player in player_ids:
        url = f"https://cricbuzz-cricket.p.rapidapi.com/stats/v1/player/{player}/batting"
        headers = {
            "x-rapidapi-key": "3eaece787amshfe1adf78b14d67bp1c781fjsnebfd830416ef",
            "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
        }
        
        response = requests.get(url, headers=headers)
        data = response.json()
        
        player_name = data['appIndex']['seoTitle'].split(" Profile")[0]
        headers_list = data['headers']  # ["ROWHEADER","Test","ODI","T20","IPL"]
        values = data['values']

        # Loop through formats (skip first column, which is row header)
        for col_index in range(1, len(headers_list)):
            format_name = headers_list[col_index]
            matches = innings = runs = avg = hundreds = None

            for row in values:
                stat_name = row['values'][0]
                stat_value = row['values'][col_index]  # value for this format

                if stat_name == "Matches":
                    matches = stat_value
                elif stat_name == "Innings":
                    innings = stat_value
                elif stat_name == "Runs":
                    runs = stat_value
                elif stat_name == "Average":
                    avg = stat_value
                elif stat_name == "100s":
                    hundreds = stat_value

            # Insert into MySQL
            player_tuple = (player, player_name, format_name, matches, innings, runs, avg, hundreds)
            insert_query = """
            INSERT INTO player_batting_stats
            (player_id, player_name, format, matches, innings, runs, batting_average, centuries)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
            ON DUPLICATE KEY UPDATE
            player_name=VALUES(player_name),
            matches=VALUES(matches),
            innings=VALUES(innings),
            runs=VALUES(runs),
            batting_average=VALUES(batting_average),
            centuries=VALUES(centuries);
            """
            cursor.execute(insert_query, player_tuple)
            print("inserted")
            conn.commit()

    cursor.close()
    conn.close()


inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted
inserted


In [13]:
import requests
import mysql.connector

# ---------------------------
# MySQL connection
# ---------------------------
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="2741",
    database="cricketdb"
)
cursor = conn.cursor()

# ---------------------------
# Create all_rounders_stats table
# ---------------------------
create_table_query = """
CREATE TABLE IF NOT EXISTS all_rounders_stats (
    player_id INT,
    player_name VARCHAR(100),
    format VARCHAR(10),
    runs INT,
    batting_average DECIMAL(5,2),
    centuries INT,
    wickets INT,
    bowling_average DECIMAL(5,2),
    economy DECIMAL(5,2),
    PRIMARY KEY(player_id, format)
);
"""
cursor.execute(create_table_query)
conn.commit()

# ---------------------------
# Headers for API
# ---------------------------
headers = {
    "x-rapidapi-key": "3eaece787amshfe1adf78b14d67bp1c781fjsnebfd830416ef",
    "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
}

# ---------------------------
# Step 1: Fetch top batters and bowlers (ODI)
# ---------------------------
def get_top_players(statsType, matchType="2"):
    url = "https://cricbuzz-cricket.p.rapidapi.com/stats/v1/topstats/0"
    querystring = {"statsType": statsType, "matchType": matchType}
    response = requests.get(url, headers=headers, params=querystring)
    data = response.json()
    player_ids = [player['values'][0] for player in data['values']]
    return player_ids

top_batters = get_top_players("mostRuns")
top_bowlers = get_top_players("mostWickets")

# ---------------------------
# Step 2: Identify potential all-rounders
# ---------------------------
all_rounder_ids = list(set(top_batters) & set(top_bowlers))
print(f"Found {len(all_rounder_ids)} potential all-rounders")

# ---------------------------
# Step 3: Fetch full stats and filter
# ---------------------------
for player_id in all_rounder_ids:
    # Fetch batting stats
    bat_url = f"https://cricbuzz-cricket.p.rapidapi.com/stats/v1/player/{player_id}/batting"
    bat_data = requests.get(bat_url, headers=headers).json()
    
    player_name = bat_data['appIndex']['seoTitle'].split(" Profile")[0]
    headers_list = bat_data['headers']  # ["ROWHEADER","Test","ODI","T20","IPL"]
    values = bat_data['values']
    
    # Fetch bowling stats
    bowl_url = f"https://cricbuzz-cricket.p.rapidapi.com/stats/v1/player/{player_id}/bowling"
    bowl_data = requests.get(bowl_url, headers=headers).json()
    bowl_values = bowl_data['values']
    
    # Loop through formats
    for col_index in range(1, len(headers_list)):
        format_name = headers_list[col_index]
        
        # Batting stats
        runs = avg = hundreds = None
        for row in values:
            stat_name = row['values'][0]
            stat_value = row['values'][col_index]
            if stat_name == "Runs":
                runs = int(stat_value)
            elif stat_name == "Average":
                avg = float(stat_value)
            elif stat_name == "100s":
                hundreds = int(stat_value)
        
        # Bowling stats
        wickets = bowl_avg = econ = None
        for row in bowl_values:
            stat_name = row['values'][0]
            stat_value = row['values'][col_index]
            if stat_name == "Wickets":
                wickets = int(stat_value)
            elif stat_name == "Average":
                bowl_avg = float(stat_value)
            elif stat_name == "Econ":
                econ = float(stat_value)
        
        # Filter: runs >= 1000 and wickets >= 50
        if runs is not None and wickets is not None and runs >= 1000 and wickets >= 50:
            player_tuple = (player_id, player_name, format_name, runs, avg, hundreds, wickets, bowl_avg, econ)
            
            insert_query = """
            INSERT INTO all_rounders_stats
            (player_id, player_name, format, runs, batting_average, centuries, wickets, bowling_average, economy)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
            ON DUPLICATE KEY UPDATE
            player_name=VALUES(player_name),
            runs=VALUES(runs),
            batting_average=VALUES(batting_average),
            centuries=VALUES(centuries),
            wickets=VALUES(wickets),
            bowling_average=VALUES(bowling_average),
            economy=VALUES(economy);
            """
            cursor.execute(insert_query, player_tuple)
            conn.commit()
            print(f"Inserted all-rounder: {player_name} ({format_name})")

cursor.close()
conn.close()
print("Done!")


Found 2 potential all-rounders
Inserted all-rounder: Sanath Jayasuriya (Test)
Inserted all-rounder: Sanath Jayasuriya (ODI)
Inserted all-rounder: Jacques Kallis (Test)
Inserted all-rounder: Jacques Kallis (ODI)
Inserted all-rounder: Jacques Kallis (IPL)
Done!


In [ ]:
def all_rounder_stats():
    import requests
    import mysql.connector

    # MySQL connection
    conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="2741",
        database="cricketdb"
    )
    cursor = conn.cursor()

    # Create table for all-rounders
    create_table_query = """
    CREATE TABLE IF NOT EXISTS all_rounders_stats (
        player_id INT,
        player_name VARCHAR(100),
        format VARCHAR(10),
        runs INT,
        batting_average DECIMAL(5,2),
        centuries INT,
        wickets INT,
        bowling_average DECIMAL(5,2),
        economy DECIMAL(5,2),
        PRIMARY KEY(player_id, format)
    );
    """
    cursor.execute(create_table_query)
    conn.commit()
    cursor.execute("SELECT player_id FROM players where role like '% Allrounder'")
    player_ids = [row[0] for row in cursor.fetchall()]

    headers = {
        "x-rapidapi-key": "3eaece787amshfe1adf78b14d67bp1c781fjsnebfd830416ef",
        "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
    }

    for player_id in player_ids:

        try:
            # Fetch batting stats
            bat_url = f"https://cricbuzz-cricket.p.rapidapi.com/stats/v1/player/{player_id}/batting"
            bat_data = requests.get(bat_url, headers=headers).json()
            
            # Skip if 'appIndex' is missing
            if 'appIndex' not in bat_data or 'values' not in bat_data:
                print(f"Skipping player {player_id} – no batting data")
                continue

            # Fetch bowling stats
            bowl_url = f"https://cricbuzz-cricket.p.rapidapi.com/stats/v1/player/{player_id}/bowling"
            bowl_data = requests.get(bowl_url, headers=headers).json()

            if 'values' not in bowl_data:
                print(f"Skipping player {player_id} – no bowling data")
                continue

            # Player name
            player_name = bat_data['appIndex']['seoTitle'].split(" Profile")[0]

            headers_list = bat_data['headers']  # ["ROWHEADER","Test","ODI","T20","IPL"]
            bat_values = bat_data['values']
            bowl_values = bowl_data['values']

            # Process each format
            for col_index in range(1, len(headers_list)):
                format_name = headers_list[col_index]

                # Batting stats
                runs = avg = hundreds = None
                for row in bat_values:
                    if row['values'][0] == "Runs":
                        runs = int(row['values'][col_index])
                    elif row['values'][0] == "Average":
                        avg = float(row['values'][col_index])
                    elif row['values'][0] == "100s":
                        hundreds = int(row['values'][col_index])

                # Bowling stats
                wickets = bowl_avg = econ = None
                for row in bowl_values:
                    if row['values'][0] == "Wickets":
                        wickets = int(row['values'][col_index])
                    elif row['values'][0] == "Average":
                        bowl_avg = float(row['values'][col_index])
                    elif row['values'][0] == "Econ":
                        econ = float(row['values'][col_index])

                # Filter all-rounders
                player_tuple = (player_id, player_name, format_name, runs, wickets)
                insert_query = """
                INSERT INTO all_rounders_stats
                (player_id, player_name, format, runs, wickets)
                VALUES (%s, %s, %s, %s, %s)
                ON DUPLICATE KEY UPDATE
                player_name=VALUES(player_name),
                runs=VALUES(runs),
                wickets=VALUES(wickets);
                """
                cursor.execute(insert_query, player_tuple)
                conn.commit()
                print(f"Inserted stats: {player_name} ({format_name})")

        except Exception as e:
            print(f"Error for player {player_id}: {e}")
    #SELECT player_name, SUM(runs) AS total_runs, SUM(wickets) AS total_wickets
    # FROM all_rounders_stats
    # GROUP BY player_name

    cursor.close()
    conn.close()
    print("Done!")


Inserted stats: Ragheb Aga (Test)
Inserted stats: Ragheb Aga (ODI)
Inserted stats: Ragheb Aga (T20)
Inserted stats: Ragheb Aga (IPL)
Inserted stats: Sean Williams (Test)
Inserted stats: Sean Williams (ODI)
Inserted stats: Sean Williams (T20)
Inserted stats: Sean Williams (IPL)
Inserted stats: Collins Obuya (Test)
Inserted stats: Collins Obuya (ODI)
Inserted stats: Collins Obuya (T20)
Inserted stats: Collins Obuya (IPL)
Inserted stats: Nehemiah Odhiambo (Test)
Inserted stats: Nehemiah Odhiambo (ODI)
Inserted stats: Nehemiah Odhiambo (T20)
Inserted stats: Nehemiah Odhiambo (IPL)
Inserted stats: Ravindra Jadeja (Test)
Inserted stats: Ravindra Jadeja (ODI)
Inserted stats: Ravindra Jadeja (T20)
Inserted stats: Ravindra Jadeja (IPL)
Inserted stats: Paul Stirling (Test)
Inserted stats: Paul Stirling (ODI)
Inserted stats: Paul Stirling (T20)
Inserted stats: Paul Stirling (IPL)
Inserted stats: Mahmudullah (Test)
Inserted stats: Mahmudullah (ODI)
Inserted stats: Mahmudullah (T20)
Inserted stats:

In [4]:
import requests
player_id = 25

headers = {
    "x-rapidapi-key": "3eaece787amshfe1adf78b14d67bp1c781fjsnebfd830416ef",
    "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
}

bat_url = f"https://cricbuzz-cricket.p.rapidapi.com/stats/v1/player/{player_id}/batting"
bat_data = requests.get(bat_url, headers=headers).json()
print(bat_data)

{'headers': ['ROWHEADER', 'Test', 'ODI', 'T20', 'IPL'], 'values': [{'values': ['Matches', '200', '463', '1', '78']}, {'values': ['Innings', '329', '452', '1', '78']}, {'values': ['Runs', '15921', '18426', '10', '2334']}, {'values': ['Balls', '29437', '21367', '12', '1948']}, {'values': ['Highest', '248', '200', '10', '100']}, {'values': ['Average', '53.79', '44.83', '10', '33.83']}, {'values': ['SR', '54.09', '86.24', '83.34', '119.82']}, {'values': ['Not Out', '33', '41', '0', '9']}, {'values': ['Fours', '2058', '2016', '2', '295']}, {'values': ['Sixes', '69', '195', '0', '29']}, {'values': ['Ducks', '14', '20', '0', '4']}, {'values': ['50s', '68', '96', '0', '13']}, {'values': ['100s', '51', '49', '0', '1']}, {'values': ['200s', '6', '1', '0', '0']}, {'values': ['300s', '0', '0', '0', '0']}, {'values': ['400s', '0', '0', '0', '0']}], 'appIndex': {'seoTitle': 'Sachin Tendulkar Profile - Cricbuzz | Cricbuzz.com', 'webURL': 'http://www.cricbuzz.com/profiles/25/sachin-tendulkar'}, 'serie

In [46]:
import requests

url = "https://cricbuzz-cricket.p.rapidapi.com/series/v1/archives/international"

querystring = {"year":"2024"}

headers = {
	"x-rapidapi-key": "3eaece787amshfe1adf78b14d67bp1c781fjsnebfd830416ef",
	"x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

print(response.json())

{'seriesMapProto': [{'date': '2024', 'series': [{'id': 8553, 'name': 'Sri Lanka tour of New Zealand, 2024-25', 'startDt': '1734912000000', 'endDt': '1736553600000'}, {'id': 9297, 'name': 'Gulf Cricket T20I Championship, 2024', 'startDt': '1734048000000', 'endDt': '1734739200000'}, {'id': 8044, 'name': 'Pakistan tour of South Africa, 2024 -25', 'startDt': '1733788800000', 'endDt': '1736208000000'}, {'id': 9171, 'name': 'Afghanistan tour of Zimbabwe, 2024-25', 'startDt': '1733702400000', 'endDt': '1736121600000'}, {'id': 9265, 'name': 'ICC Mens T20 World Cup Americas Sub Regional Qualifier, 2024', 'startDt': '1733443200000', 'endDt': '1734307200000'}, {'id': 9289, 'name': 'Africa Continental Cup 2024', 'startDt': '1733270400000', 'endDt': '1734134400000'}, {'id': 8058, 'name': 'Sri Lanka tour of South Africa, 2024', 'startDt': '1732665600000', 'endDt': '1733702400000'}, {'id': 8494, 'name': 'Pakistan tour of Zimbabwe, 2024', 'startDt': '1732406400000', 'endDt': '1733356800000', 'isFantas

In [47]:
def update_series_table_2024():
    import requests
    import mysql.connector
    from datetime import datetime

    # -------------------
    # Database connection
    # -------------------
    conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="2741",
        database="cricketdb"
    )
    cursor = conn.cursor()

    # -------------------
    # Create series table if not exists
    # -------------------
    create_table_query = """
    CREATE TABLE IF NOT EXISTS series_2024 (
        series_id INT PRIMARY KEY,
        series_name VARCHAR(255),
        host_country VARCHAR(100),
        start_date DATE,
        end_date DATE,
        total_matches INT
    )
    """
    cursor.execute(create_table_query)
    conn.commit()

    # -------------------
    # RapidAPI config
    # -------------------
    API_KEY = "3eaece787amshfe1adf78b14d67bp1c781fjsnebfd830416ef"
    HEADERS = {
        "x-rapidapi-key": API_KEY,
        "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
    }

    # -------------------
    # Fetch series list
    # -------------------
    series_url = "https://cricbuzz-cricket.p.rapidapi.com/series/v1/archives/international"

    querystring = {"year":"2024"}

    try:
        series_response = requests.get(series_url, headers=HEADERS, params=querystring, timeout = 10)
        series_response.raise_for_status()
        series_data = series_response.json()
    except requests.RequestException as e:
        print(f"Error fetching series list: {e}")
        series_data = {}
    except ValueError as e:  # JSON decode error
        print(f"Error parsing series JSON: {e}")
        series_data = {}

    # -------------------
    # Helper: get total matches for a series
    # -------------------
    def get_total_matches(series_id):
        url = f"https://cricbuzz-cricket.p.rapidapi.com/series/v1/{series_id}"
        try:
            response = requests.get(url, headers=HEADERS, timeout=10)
            response.raise_for_status()
            data = response.json()
        except requests.RequestException as e:
            print(f"Error fetching matches for series {series_id}: {e}")
            return 0
        except ValueError as e:
            print(f"Error parsing matches JSON for series {series_id}: {e}")
            return 0

        total_matches = 0
        for item in data.get("matchDetails", []):
            match_map = item.get("matchDetailsMap")
            if match_map and "match" in match_map:
                total_matches += len(match_map["match"])
        return total_matches

    # -------------------
    # Insert series data
    # -------------------
    for year_block in series_data.get("seriesMapProto", []):
        for s in year_block.get("series", []):
            series_id = s["id"]
            series_name = s["name"]
            start_date = datetime.fromtimestamp(int(s["startDt"]) / 1000).strftime("%Y-%m-%d")
            end_date = datetime.fromtimestamp(int(s["endDt"]) / 1000).strftime("%Y-%m-%d")

            # Fetch host country from venues
            venue_url = f"https://cricbuzz-cricket.p.rapidapi.com/series/v1/{series_id}/venues"
            try:
                venue_response = requests.get(venue_url, headers=HEADERS, timeout=10)
                venue_response.raise_for_status()
                venue_data = venue_response.json()
                venues = venue_data.get("seriesVenue", [])
                host_country = venues[0].get("country") if venues else None
            except requests.RequestException as e:
                print(f"Warning: Could not fetch venues for series {series_id}: {e}")
                host_country = None
            except ValueError as e:
                print(f"Warning: Could not parse venue JSON for series {series_id}: {e}")
                host_country = None

            # Get total matches from matches API
            total_matches = get_total_matches(series_id)

            # Insert into DB
            insert_query = """
            INSERT INTO series_2024 (series_id, series_name, host_country, start_date, end_date, total_matches)
            VALUES (%s, %s, %s, %s, %s, %s)
            ON DUPLICATE KEY UPDATE
                series_name = VALUES(series_name),
                host_country = VALUES(host_country),
                start_date = VALUES(start_date),
                end_date = VALUES(end_date),
                total_matches = VALUES(total_matches)
            """
            cursor.execute(insert_query, (series_id, series_name, host_country, start_date, end_date, total_matches))
            conn.commit()
            print(f"Inserted/Updated series {series_name} with {total_matches} matches.")

    cursor.close()
    conn.close()
    print("Series table updated successfully!")

In [48]:
update_series_table_2024()

Inserted/Updated series Sri Lanka tour of New Zealand, 2024-25 with 8 matches.
Inserted/Updated series Gulf Cricket T20I Championship, 2024 with 16 matches.
Inserted/Updated series Pakistan tour of South Africa, 2024 -25 with 8 matches.
Inserted/Updated series Afghanistan tour of Zimbabwe, 2024-25 with 8 matches.
Inserted/Updated series ICC Mens T20 World Cup Americas Sub Regional Qualifier, 2024 with 36 matches.
Inserted/Updated series Africa Continental Cup 2024 with 19 matches.
Inserted/Updated series Sri Lanka tour of South Africa, 2024 with 2 matches.
Inserted/Updated series Pakistan tour of Zimbabwe, 2024 with 6 matches.
Inserted/Updated series England tour of New Zealand, 2024 with 4 matches.
Inserted/Updated series ICC Mens T20 World Cup Africa Sub Regional Qualifier C 2024 with 15 matches.
Inserted/Updated series India tour of Australia, 2024-25 with 6 matches.
Inserted/Updated series ICC Mens T20 World Cup Asia Qualifier B 2024 with 21 matches.
Inserted/Updated series Banglad

In [ ]:
def player_batting_stats():  # query 11
    import requests
    import mysql.connector

    # -------------------
    # Step 1: Fetch top players (just to get IDs)
    # -------------------
    url = "https://cricbuzz-cricket.p.rapidapi.com/stats/v1/topstats/0"
    querystring = {"statsType": "mostRuns"}  # You said you'll remove matchType

    headers = {
        "x-rapidapi-key": "3eaece787amshfe1adf78b14d67bp1c781fjsnebfd830416ef",
        "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)
    player_ids = []
    data = response.json()

    for player in data.get("values", []):
        values = player["values"]
        player_id = values[0]
        player_ids.append(player_id)   # ✅ fixed

    # -------------------
    # Step 2: MySQL connection
    # -------------------
    conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="2741",
        database="cricketdb"
    )
    cursor = conn.cursor()

    # Create table if not exists
    create_table_query = """
    CREATE TABLE IF NOT EXISTS player_batting_stats (
        player_id INT,
        player_name VARCHAR(100),
        format VARCHAR(10),
        matches INT,
        innings INT,
        runs INT,
        batting_average DECIMAL(6,2),
        centuries INT,
        PRIMARY KEY(player_id, format)
    );
    """
    cursor.execute(create_table_query)
    conn.commit()

    # -------------------
    # Step 3: Loop through players → fetch stats
    # -------------------
    for player in player_ids:
        url = f"https://cricbuzz-cricket.p.rapidapi.com/stats/v1/player/{player}/batting"
        try:
            response = requests.get(url, headers=headers, timeout=10)
            response.raise_for_status()
            data = response.json()
        except Exception as e:
            print(f"⚠️ Error fetching stats for player {player}: {e}")
            continue

        if "headers" not in data or "values" not in data:
            continue

        player_name = data.get("appIndex", {}).get("seoTitle", "").split(" Profile")[0]
        headers_list = data["headers"]  # ["ROWHEADER","Test","ODI","T20I","IPL"]
        values = data["values"]

        # Loop through formats
        for col_index in range(1, len(headers_list)):
            format_name = headers_list[col_index]

            # ✅ Optional: skip IPL if you only want international
            if format_name not in ["Test", "ODI", "T20I"]:
                continue

            matches = innings = runs = avg = hundreds = None

            for row in values:
                stat_name = row["values"][0]
                stat_value = row["values"][col_index]

                if stat_value in ["-", "", None]:
                    continue

                if stat_name == "Matches":
                    matches = int(stat_value)
                elif stat_name == "Innings":
                    innings = int(stat_value)
                elif stat_name == "Runs":
                    runs = int(stat_value)
                elif stat_name == "Average":
                    try:
                        avg = float(stat_value)
                    except:
                        avg = None
                elif stat_name == "100s":
                    hundreds = int(stat_value)

            # Insert into DB
            player_tuple = (player, player_name, format_name, matches, innings, runs, avg, hundreds)
            insert_query = """
            INSERT INTO player_batting_stats
            (player_id, player_name, format, matches, innings, runs, batting_average, centuries)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
            ON DUPLICATE KEY UPDATE
            player_name=VALUES(player_name),
            matches=VALUES(matches),
            innings=VALUES(innings),
            runs=VALUES(runs),
            batting_average=VALUES(batting_average),
            centuries=VALUES(centuries);
            """
            cursor.execute(insert_query, player_tuple)
            conn.commit()
            print(f"✅ Inserted/Updated {player_name} ({format_name})")

    cursor.close()
    conn.close()
    print("All player batting stats updated successfully!")
#SELECT 
#     p.player_id,
#     p.player_name,
#     MAX(CASE WHEN p.format = 'Test' THEN p.runs ELSE 0 END) AS Test_Runs,
#     MAX(CASE WHEN p.format = 'ODI' THEN p.runs ELSE 0 END) AS ODI_Runs,
#     MAX(CASE WHEN p.format = 'T20I' THEN p.runs ELSE 0 END) AS T20I_Runs,
#     ROUND(AVG(p.batting_average), 2) AS Overall_Batting_Avg,
#     COUNT(DISTINCT p.format) AS Formats_Played
# FROM 
#     player_batting_stats p
# GROUP BY 
#     p.player_id, p.player_name
# HAVING 
#     Formats_Played >= 2
# ORDER BY 
#     Overall_Batting_Avg DESC;


In [50]:
player_batting_stats()

All player batting stats updated successfully!


[{'match_id': 130168, 'innings_id': 1, 'bat1id': 12086, 'bat1name': 'Abhishek Sharma', 'bat2id': 11808, 'bat2name': 'Gill', 'total_runs': 15, 'total_balls': 9}, {'match_id': 130168, 'innings_id': 1, 'bat1id': 12086, 'bat1name': 'Abhishek Sharma', 'bat2id': 7915, 'bat2name': 'Suryakumar Yadav', 'total_runs': 59, 'total_balls': 32}, {'match_id': 130168, 'innings_id': 1, 'bat1id': 12086, 'bat1name': 'Abhishek Sharma', 'bat2id': 14504, 'bat2name': 'Tilak Varma', 'total_runs': 18, 'total_balls': 11}, {'match_id': 130168, 'innings_id': 1, 'bat1id': 8271, 'bat1name': 'Samson', 'bat2id': 14504, 'bat2name': 'Tilak Varma', 'total_runs': 66, 'total_balls': 41}, {'match_id': 130168, 'innings_id': 1, 'bat1id': 9647, 'bat1name': 'Hardik Pandya', 'bat2id': 14504, 'bat2name': 'Tilak Varma', 'total_runs': 4, 'total_balls': 4}, {'match_id': 130168, 'innings_id': 1, 'bat1id': 8808, 'bat1name': 'Axar', 'bat2id': 14504, 'bat2name': 'Tilak Varma', 'total_runs': 40, 'total_balls': 23}, {'match_id': 130168, '

In [ ]:
def query_13():
    import mysql.connector

    import requests

    url = "https://cricbuzz-cricket.p.rapidapi.com/matches/v1/recent"

    headers = {
        "x-rapidapi-key": "3eaece787amshfe1adf78b14d67bp1c781fjsnebfd830416ef",
        "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers)
    data = response.json()

    match_ids = []
    partnership_data = []

    for matches in data.get("typeMatches", []):
        for series in matches.get("seriesMatches", []):
            wrapper = series.get("seriesAdWrapper")
            if wrapper:
                for match in wrapper.get('matches', []):
                    info = match.get("matchInfo", {})
                    match_id = info.get("matchId")
                    if match_id:
                        match_ids.append(match_id)

    for p in match_ids:
        scurl = f"https://cricbuzz-cricket.p.rapidapi.com/mcenter/v1/{p}/hscard"
        response = requests.get(scurl, headers=headers)
        res = response.json()
        
        for innings in res.get("scorecard", []):   # check spelling: 'scorcard' or 'scorecard'?
            innings_id = innings.get("inningsid")
            partnerships = innings.get("partnership", {}).get("partnership", [])
            for data in partnerships:
                bat1id = data.get("bat1id")
                bat1name = data.get("bat1name")
                bat2id = data.get("bat2id")
                bat2name = data.get("bat2name")
                total_runs = data.get("totalruns")
                balls_faced = data.get("totalballs")

                partnership_data.append({
                    "match_id": p,
                    "innings_id": innings_id,
                    "bat1id": bat1id,
                    "bat1name": bat1name,
                    "bat2id": bat2id,
                    "bat2name": bat2name,
                    "total_runs": total_runs,
                    "total_balls": balls_faced
                })


    # --- Connect to MySQL ---
    conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="2741",
        database="cricketdb"
    )

    cursor = conn.cursor()

    # --- Create table ---
    create_table_query = """
    CREATE TABLE IF NOT EXISTS partnerships (
        id INT AUTO_INCREMENT PRIMARY KEY,
        match_id BIGINT NOT NULL,
        innings_id INT NOT NULL,
        bat1id BIGINT,
        bat1name VARCHAR(100),
        bat2id BIGINT,
        bat2name VARCHAR(100),
        total_runs INT,
        total_balls INT
    );
    """

    cursor.execute(create_table_query)
    conn.commit()

    # --- Insert data ---
    insert_query = """
    INSERT INTO partnerships (
        match_id, innings_id, bat1id, bat1name, bat2id, bat2name, total_runs, total_balls
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    """

    # Example: partnership_data is the list of dictionaries you collected
    for p in partnership_data:
        cursor.execute(insert_query, (
            p["match_id"],
            p["innings_id"],
            p.get("bat1id"),
            p.get("bat1name"),
            p.get("bat2id"),
            p.get("bat2name"),
            p.get("total_runs"),
            p.get("total_balls")
        ))
        conn.commit()




    print(f"{cursor.rowcount} records inserted into partnerships table.")

    # --- Close connection ---
    cursor.close()
    conn.close()


1 records inserted into partnerships table.


In [ ]:
import requests

headers = {
        "x-rapidapi-key": "3eaece787amshfe1adf78b14d67bp1c781fjsnebfd830416ef",
        "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
    }
url = "https://cricbuzz-cricket.p.rapidapi.com/matches/v1/recent"

response = requests.get(url, headers=headers)
data = response.json()

match_ids = []
for matches in data.get("typeMatches", []):
            for series in matches.get("seriesMatches", []):
                wrapper = series.get("seriesAdWrapper")
                if wrapper:
                    for match in wrapper.get('matches', []):
                        info = match.get("matchInfo", {})
                        match_id = info.get("matchId")
                        venue_id = info.get("venueInfo").get("id")
                        if match_id:
                            match_ids.append(match_id)

[{'match_id': 130157, 'player_id': 13625, 'player_name': 'Sahibzada Farhan', 'runs_scored': 4, 'team_name': 'Pakistan', 'team_won': True}, {'match_id': 130157, 'player_id': 10863, 'player_name': 'Fakhar Zaman', 'runs_scored': 13, 'team_name': 'Pakistan', 'team_won': True}, {'match_id': 130157, 'player_id': 14241, 'player_name': 'Saim Ayub', 'runs_scored': 0, 'team_name': 'Pakistan', 'team_won': True}, {'match_id': 130157, 'player_id': 10101, 'player_name': 'Salman Agha', 'runs_scored': 19, 'team_name': 'Pakistan', 'team_won': True}, {'match_id': 130157, 'player_id': 9804, 'player_name': 'Hussain Talat', 'runs_scored': 3, 'team_name': 'Pakistan', 'team_won': True}, {'match_id': 130157, 'player_id': 15038, 'player_name': 'Mohammad Haris', 'runs_scored': 31, 'team_name': 'Pakistan', 'team_won': True}, {'match_id': 130157, 'player_id': 12160, 'player_name': 'Shaheen Afridi', 'runs_scored': 19, 'team_name': 'Pakistan', 'team_won': True}, {'match_id': 130157, 'player_id': 8367, 'player_name'

In [1]:
import mysql.connector
import requests

# -------------------
# Config
# -------------------
HEADERS = {
    "x-rapidapi-key": "3eaece787amshfe1adf78b14d67bp1c781fjsnebfd830416ef",
    "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
}
RECENT_MATCHES_URL = "https://cricbuzz-cricket.p.rapidapi.com/matches/v1/recent"

# -------------------
# DB Connection
# -------------------
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="2741",
    database="cricketdb"
)
cursor = conn.cursor()

INSERT_QUERY = """
INSERT INTO match_player_stats
(match_id, player_id, player_name, runs_scored, balls_faced, fours, sixes, strike_rate, team_name, team_won, win_margin, win_by_runs, win_by_innings)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

# -------------------
# Fetch recent matches
# -------------------
resp = requests.get(RECENT_MATCHES_URL, headers=HEADERS)
data = resp.json()

for match_type in data.get("typeMatches", []):
    for series in match_type.get("seriesMatches", []):
        wrapper = series.get("seriesAdWrapper")
        if not wrapper:
            continue

        for match in wrapper.get("matches", []):
            match_info = match.get("matchInfo", {})
            match_id = match_info.get("matchId")
            if not match_id:
                continue

            # Fetch miniscore for the match
            sc_url = f"https://cricbuzz-cricket.p.rapidapi.com/mcenter/v1/{match_id}/leanback"
            sc_resp = requests.get(sc_url, headers=HEADERS)
            sc_data = sc_resp.json()
            if not sc_data:
                continue

            # -------------------
            # Match result info
            # -------------------
            match_header = sc_data.get("matchHeader", {})
            result = match_header.get("result", {})

            winning_team_id = result.get("winningteamId")
            winning_margin = result.get("winningMargin", 0)
            win_by_runs = 1 if result.get("winByRuns", True) else 0
            win_by_innings = 1 if result.get("winByInnings", False) else 0

            # -------------------
            # Player-level info
            # -------------------
            miniscore = sc_data.get("miniscore", {})
            bat_team = miniscore.get("batTeam", {})
            bat_team_id = bat_team.get("teamId")
            bat_team_name = bat_team.get("teamName", "")

            # Striker
            striker = miniscore.get("batsmanStriker", {})
            if striker:
                player_stats = (
                    match_id,
                    striker.get("batId"),
                    striker.get("batName"),
                    striker.get("batRuns", 0),
                    striker.get("batBalls", 0),
                    striker.get("batFours", 0),
                    striker.get("batSixes", 0),
                    striker.get("batStrikeRate", 0.0),
                    bat_team_name,
                    1 if bat_team_id == winning_team_id else 0,
                    winning_margin,
                    win_by_runs,
                    win_by_innings
                )
                cursor.execute(INSERT_QUERY, player_stats)

            # Non-striker
            non_striker = miniscore.get("batsmanNonStriker", {})
            if non_striker:
                player_stats = (
                    match_id,
                    non_striker.get("batId"),
                    non_striker.get("batName"),
                    non_striker.get("batRuns", 0),
                    non_striker.get("batBalls", 0),
                    non_striker.get("batFours", 0),
                    non_striker.get("batSixes", 0),
                    non_striker.get("batStrikeRate", 0.0),
                    bat_team_name,
                    1 if bat_team_id == winning_team_id else 0,
                    winning_margin,
                    win_by_runs,
                    win_by_innings
                )
                cursor.execute(INSERT_QUERY, player_stats)

            conn.commit()
            print(f"Inserted players for match {match_id} — winner team id: {winning_team_id}, margin: {winning_margin}, innings win: {win_by_innings}")

cursor.close()
conn.close()


Inserted players for match 130168 — winner team id: None, margin: 0, innings win: 0
Inserted players for match 130129 — winner team id: None, margin: 0, innings win: 0
Inserted players for match 130140 — winner team id: None, margin: 0, innings win: 0
Inserted players for match 130157 — winner team id: None, margin: 0, innings win: 0


: 

In [1]:
import mysql.connector

# -------------------
# DB Connection
# -------------------
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="2741",
    database="cricketdb"
)
cursor = conn.cursor()

# -------------------
# Create Table Query
# -------------------
create_table_query = """
CREATE TABLE player_bowling_stats (
    id INT AUTO_INCREMENT PRIMARY KEY,
    player_id INT NOT NULL,
    player_name VARCHAR(100) NOT NULL,
    match_type VARCHAR(10) NOT NULL,  -- "odi" or "t20"
    matches INT,
    overs DECIMAL(7,2),
    wickets INT,
    economy DECIMAL(5,2)
);


"""

cursor.execute(create_table_query)
conn.commit()

cursor.close()
conn.close()

print("✅ match_player_stats table created successfully!")


✅ match_player_stats table created successfully!


In [3]:
import requests
import mysql.connector

# Connect to your MySQL
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="2741",
    database="cricketdb"
)
cursor = conn.cursor()

matchtype =[2,3]  # 2 = ODI, 3 = T20

insert_query = """
INSERT INTO player_bowling_stats
(player_id, player_name, match_type, matches, overs, wickets, economy)
VALUES (%s, %s, %s, %s, %s, %s, %s)
"""

for type in matchtype:
    url = "https://cricbuzz-cricket.p.rapidapi.com/stats/v1/topstats/0"
    querystring = {"statsType":"lowestEcon","year":"2025","matchType":f"{type}"}
    headers = {
        "x-rapidapi-key": "3eaece787amshfe1adf78b14d67bp1c781fjsnebfd830416ef",
        "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)
    bowl_stats = response.json()

    for val in bowl_stats.get('values', []):
        player_id = val.get('values')[0]
        player_name = val.get('values')[1]
        matches = val.get('values')[2]
        overs = val.get('values')[3]
        wickets = val.get('values')[4]
        economy = val.get('values')[5]
        match_name = "odi" if type == 2 else "t20"
        cursor.execute(insert_query, (
            player_id,
            player_name,
            match_name,
            matches,
            overs,
            wickets,
            economy
        ))

# Commit and close
conn.commit()
cursor.close()
conn.close()


In [ ]:
import requests
from datetime import datetime

series_ids_22_25 = [3961,4499,6906,5917,7476,9129,9315,10587]
match_ids = []  # store tuples of (matchId, matchDate)
match_dates=[]
headers = {
    "x-rapidapi-key": "3eaece787amshfe1adf78b14d67bp1c781fjsnebfd830416ef",
    "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
}

for series_id in series_ids_22_25:
    url = f"https://cricbuzz-cricket.p.rapidapi.com/series/v1/{series_id}"
    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        print(f"Series {series_id} failed with status {response.status_code}")
        continue

    try:
        seriesdata = response.json()
    except Exception as e:
        print(f"Series {series_id} JSON decode failed: {e}")
        continue

    for detail in seriesdata.get("matchDetails", []):
        match_details_map = detail.get("matchDetailsMap", {})
        if isinstance(match_details_map, list):
            mdm_list = match_details_map
        else:
            mdm_list = [match_details_map]

        for mdm in mdm_list:
            for match in mdm.get("match", []):
                match_info = match.get("matchInfo", {})
                match_id = match_info.get("matchId")
                start_date_ms = match_info.get("startDate")
                if match_id and start_date_ms:
                    # Convert milliseconds since epoch to YYYY-MM-DD
                    match_date = datetime.fromtimestamp(int(start_date_ms)/1000).strftime("%Y-%m-%d")
                    match_ids.append(match_id)
                    match_dates.append(match_date)



Series 9315 failed with status 204


In [13]:
import requests

# Example match_ids list
batsman_stats = []

headers = {
    "x-rapidapi-key": "3eaece787amshfe1adf78b14d67bp1c781fjsnebfd830416ef",
    "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
}

for match_id, match_date in zip(match_ids, match_dates):

    url = f"https://cricbuzz-cricket.p.rapidapi.com/mcenter/v1/{match_id}/hscard"
    
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(f"Match {match_id} failed with status {response.status_code}")
        continue
    
    try:
        match_data = response.json()
    except Exception as e:
        print(f"Match {match_id} JSON decode failed: {e}")
        continue
    
    # Loop through innings
    for inning in match_data.get("scorecard", []):
        for batsman in inning.get("batsman", []):
            batsman_stats.append({
                "player_id": batsman.get("id"),
                "player_name": batsman.get("name"),
                "runs": batsman.get("runs"),
                "balls": batsman.get("balls"),
                "match_id":match_id,
                "match_date":match_date

            })

# Example output
for stat in batsman_stats[:10]:  # show first 10
    print(stat)



{'player_id': 11136, 'player_name': 'Michael van Lingen', 'runs': 3, 'balls': 6, 'match_id': 42986, 'match_date': '2022-10-16'}
{'player_id': 19497, 'player_name': 'Divan la Cock', 'runs': 9, 'balls': 9, 'match_id': 42986, 'match_date': '2022-10-16'}
{'player_id': 13438, 'player_name': 'Loftie-Eaton', 'runs': 20, 'balls': 12, 'match_id': 42986, 'match_date': '2022-10-16'}
{'player_id': 7971, 'player_name': 'Baard', 'runs': 26, 'balls': 24, 'match_id': 42986, 'match_date': '2022-10-16'}
{'player_id': 7973, 'player_name': 'Erasmus', 'runs': 20, 'balls': 24, 'match_id': 42986, 'match_date': '2022-10-16'}
{'player_id': 8522, 'player_name': 'Jan Frylinck', 'runs': 44, 'balls': 28, 'match_id': 42986, 'match_date': '2022-10-16'}
{'player_id': 8589, 'player_name': 'Wiese', 'runs': 0, 'balls': 1, 'match_id': 42986, 'match_date': '2022-10-16'}
{'player_id': 9885, 'player_name': 'Smit', 'runs': 31, 'balls': 16, 'match_id': 42986, 'match_date': '2022-10-16'}
{'player_id': 9890, 'player_name': 'Zan

In [11]:
import mysql.connector

# -------------------
# DB Connection
# -------------------
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="2741",
    database="cricketdb"
)
cursor = conn.cursor()

# -------------------
# Create Table Query
# -------------------
create_table_query = """
CREATE TABLE batsman_stats_22_25 (
    id INT AUTO_INCREMENT PRIMARY KEY,
    player_id INT NOT NULL,
    player_name VARCHAR(100) NOT NULL,
    runs INT NOT NULL,
    balls INT NOT NULL,
    match_date DATE NOT NULL,
    match_id INT NOT NULL
);


"""

cursor.execute(create_table_query)
conn.commit()

cursor.close()
conn.close()

print("✅ match_player_stats table created successfully!")


✅ match_player_stats table created successfully!


In [14]:
import mysql.connector

# Connect to your DB
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="2741",
    database="cricketdb"
)
cursor = conn.cursor()

insert_query = """
INSERT INTO batsman_stats_22_25 (player_id, player_name, runs, balls, match_date, match_id)
VALUES (%s, %s, %s, %s, %s, %s)
"""

for stat in batsman_stats:
    cursor.execute(insert_query, (
        stat["player_id"],
        stat["player_name"],
        stat["runs"],
        stat["balls"],
        stat["match_date"],
        stat["match_id"]
    ))

conn.commit()
cursor.close()
conn.close()


In [4]:
import requests
import mysql.connector

# ----------- DB Connection -----------
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="2741",
    database="cricketdb"
)
cursor = conn.cursor()

# ----------- Get 100 players from players table -----------
cursor.execute("SELECT player_id, player_name FROM players LIMIT 100")
players = cursor.fetchall()  # List of tuples: (player_id, player_name)

# ----------- API Config -----------
headers = {
    "x-rapidapi-key": "3eaece787amshfe1adf78b14d67bp1c781fjsnebfd830416ef",
    "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
}

formats = ["Test", "ODI", "T20", "IPL"]

# ----------- Fetch Batting Stats & Insert -----------
for pid, pname in players:
    BATTING_API = f"https://cricbuzz-cricket.p.rapidapi.com/stats/v1/player/{pid}/batting"
    response = requests.get(BATTING_API, headers=headers)
    if response.status_code != 200:
        print(f"Batting API failed for {pname}")
        continue
    data = response.json()

    # Build mapping from row headers to format values
    batting_map = {}
    for row in data['values']:
        stat_name = row['values'][0]
        if stat_name in ["Matches", "Innings", "Runs", "Balls", "Average", "SR"]:
            batting_map[stat_name] = row['values'][1:]  # values per format

    for i, fmt in enumerate(formats):
        insert_batting = """
        INSERT INTO batting_stats 
        (player_id, player_name, format, matches, innings, runs, balls, average, strike_rate)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        cursor.execute(insert_batting, (
            pid,
            pname,
            fmt,
            int(batting_map.get("Matches")[i]),
            int(batting_map.get("Innings")[i]),
            int(batting_map.get("Runs")[i]),
            int(batting_map.get("Balls")[i]),
            float(batting_map.get("Average")[i]),
            float(batting_map.get("SR")[i])
        ))

# ----------- Fetch Bowling Stats & Insert -----------
for pid, pname in players:
    BOWLING_API = f"https://cricbuzz-cricket.p.rapidapi.com/stats/v1/player/{pid}/bowling"
    response = requests.get(BOWLING_API, headers=headers)
    if response.status_code != 200:
        print(f"Bowling API failed for {pname}")
        continue
    data = response.json()

    bowling_map = {}
    for row in data['values']:
        stat_name = row['values'][0]
        if stat_name in ["Matches", "Wickets", "Avg", "Eco"]:
            bowling_map[stat_name] = row['values'][1:]

    for i, fmt in enumerate(formats):
        insert_bowling = """
        INSERT INTO bowling_stats
        (player_id, player_name, format, matches, wickets, average, economy)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        """
        cursor.execute(insert_bowling, (
            pid,
            pname,
            fmt,
            int(bowling_map.get("Matches")[i]),
            int(bowling_map.get("Wickets")[i]),
            float(bowling_map.get("Avg")[i]),
            float(bowling_map.get("Eco")[i])
        ))

# ----------- Commit & Close -----------
conn.commit()
cursor.close()
conn.close()

print("Batting and bowling stats fetched and inserted correctly for 100 players.")


Batting and bowling stats fetched and inserted correctly for 100 players.


In [19]:
def update_match_table():
    import mysql.connector
    import requests
    from datetime import datetime

    # -----------------------
    # Database connection
    # -----------------------
    conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="2741",
        database="cricketdb"
    )
    cursor = conn.cursor()

    # -----------------------
    # Fetch latest matches
    # -----------------------
    def fetch_latest_matches():
        url = "https://cricbuzz-cricket.p.rapidapi.com/matches/v1/recent"
        headers = {
            "X-RapidAPI-Key": "3eaece787amshfe1adf78b14d67bp1c781fjsnebfd830416ef",
            "X-RapidAPI-Host": "cricbuzz-cricket.p.rapidapi.com"
        }
        response = requests.get(url, headers=headers)
        return response.json() if response.status_code == 200 else {}

    # -----------------------
    # Batch insert helper
    # -----------------------
    def batch_insert(cursor, query, data_list, batch_size=20):
        for i in range(0, len(data_list), batch_size):
            batch = data_list[i:i+batch_size]
            cursor.executemany(query, batch)
            conn.commit()  # commit per batch

    # -----------------------
    # Toss info
    # -----------------------
    def toss_decisions(match_id):
        url = f"https://cricbuzz-cricket.p.rapidapi.com/mcenter/v1/{match_id}"
        headers = {
            "x-rapidapi-key": "3eaece787amshfe1adf78b14d67bp1c781fjsnebfd830416ef",
            "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
        }
        response = requests.get(url, headers=headers)
        data = response.json()

        team1 = data.get("team1", {})
        team2 = data.get("team2", {})
        toss_status = data.get("tossstatus", "")

        toss_winner_id = None
        toss_decision = None

        if toss_status:
            parts = toss_status.split(" opt to ")
            if len(parts) == 2:
                toss_winner_name = parts[0].strip()
                toss_decision = parts[1].strip().lower()
                if toss_winner_name == team1.get("teamname"):
                    toss_winner_id = team1.get("teamid")
                elif toss_winner_name == team2.get("teamname"):
                    toss_winner_id = team2.get("teamid")

        return toss_winner_id, toss_decision

    # -----------------------
    # Main loop: collect all data
    # -----------------------
    data = fetch_latest_matches()
    all_teams = set()
    all_venues = set()
    all_matches = []
    venues_ids = []

    for category in data.get("typeMatches", []):
        for series in category.get("seriesMatches", []):
            series_wrapper = series.get("seriesAdWrapper", {})
            series_id = series_wrapper.get("seriesId")
            series_name = series_wrapper.get("seriesName")

            for match in series_wrapper.get("matches", []):
                info = match.get("matchInfo", {})
                match_id = info.get("matchId")
                match_date_ts = info.get("startDate")
                match_date = datetime.fromtimestamp(int(match_date_ts)/1000).strftime("%Y-%m-%d %H:%M:%S") if match_date_ts else None
                description = info.get("matchDesc")
                venue_id = info.get("venueInfo", {}).get("id")
                venue_name = info.get("venueInfo", {}).get("ground")
                venues_ids.append(venue_id)

                team1 = info.get("team1", {})
                team2 = info.get("team2", {})

                team1_id = team1.get("teamId")
                team2_id = team2.get("teamId")

                # Winner info
                winner_team_id = None
                victory_margin = None
                victory_type = "NR"
                status = info.get("status", "")
                if "won by" in status:
                    parts = status.split(" won by ")
                    winner_team_name = parts[0].strip()
                    winner_team_id = team1_id if winner_team_name == info.get("team1", {}).get("teamName") else team2_id
                    margin_parts = parts[1].split(" ")
                    try:
                        victory_margin = int(margin_parts[0])
                        victory_type = margin_parts[1] if margin_parts[1].lower() in ["runs", "wickets"] else "NR"
                    except:
                        victory_margin = None

                # Toss info
                toss_winner_id, toss_decision = toss_decisions(match_id)

                # Collect teams
                for team in [team1, team2]:
                    t_id = team.get("teamId")
                    t_name = team.get("teamName")
                    if t_id and t_name:
                        all_teams.add((t_id, t_name))

                # Collect venues
                if venue_id and venue_name:
                    all_venues.add((venue_id, venue_name))

                # Collect matches
                all_matches.append((
                    match_id, series_id, match_date, description, venue_id,
                    team1_id, team2_id, winner_team_id, victory_margin, victory_type,
                    toss_winner_id, toss_decision
                ))

                print("Collected:", description)

    # ------------------------
    # Insert all data safely
    # ------------------------
    cursor.execute("SET FOREIGN_KEY_CHECKS = 0;")  # disable temporarily

    if all_teams:
        batch_insert(cursor, "INSERT IGNORE INTO teams (team_id, team_name) VALUES (%s, %s)", list(all_teams))
    if all_venues:
        batch_insert(cursor, "INSERT IGNORE INTO venues (venue_id, venue_name) VALUES (%s, %s)", list(all_venues))
    if all_matches:
        batch_insert(cursor, """
            INSERT INTO matches
            (match_id, series_id, match_date, description, venue_id, team1_id, team2_id,
             winner_team_id, victory_margin, victory_type, toss_winner_id, toss_decision)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            ON DUPLICATE KEY UPDATE
                series_id=VALUES(series_id), match_date=VALUES(match_date), description=VALUES(description),
                venue_id=VALUES(venue_id), team1_id=VALUES(team1_id), team2_id=VALUES(team2_id),
                winner_team_id=VALUES(winner_team_id), victory_margin=VALUES(victory_margin),
                victory_type=VALUES(victory_type), toss_winner_id=VALUES(toss_winner_id),
                toss_decision=VALUES(toss_decision)
        """, all_matches, batch_size=20)

    cursor.execute("SET FOREIGN_KEY_CHECKS = 1;")  # enable back

    print("All matches processed successfully.")
    update_venues_tables(venues_ids, conn)


In [20]:
update_match_table()

Collected: Super Fours, 18th Match (A1 v B1)
Collected: Super Fours, 16th Match (A1 v B2)
Collected: Super Fours, 15th Match (A2 v B1)
Collected: Super Fours, 17th Match (A2 v B2)
Collected: Super Fours, 14th Match (A1 v A2)
Collected: 1st T20I
Collected: 3rd Match, Group B
Collected: 1st Match, Group A
Collected: 2nd Match, Group A
Collected: 4th Match, Group B
Collected: 3rd T20I
Collected: Final
Collected: 3rd Youth ODI
Collected: 2nd Youth ODI
Collected: 1st Youth ODI
Collected: 6th Match
Collected: 5th Match
Collected: 2nd Unofficial Test
Collected: 69th Match
Collected: 66th Match
Collected: 68th Match
Collected: 70th Match
Collected: 67th Match
Collected: 53rd Match
Collected: 54th Match
Collected: 55th Match
Collected: 56th Match
Collected: 1st ODI
Collected: 3rd Match
Collected: 5th Match
Collected: Final
Collected: 4th Match
Collected: 3rd place play-off
Collected: 6th Match
Collected: 5th Match
Collected: 3rd Match
Collected: 2nd Match
Collected: 6th Match
Collected: 1st Mat

DatabaseError: 1205 (HY000): Lock wait timeout exceeded; try restarting transaction

In [3]:
!streamlit run pages/crud_operations.py


: 

In [1]:
def all_rounder_stats():
    import requests
    import mysql.connector

    # --- MySQL connection ---
    conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="2741",
        database="cricketdb"
    )
    cursor = conn.cursor()

    # --- Create table if not exists ---
    create_table_query = """
    CREATE TABLE IF NOT EXISTS all_rounders_stats (
        player_id INT,
        player_name VARCHAR(100),
        format VARCHAR(10),
        runs INT,
        batting_average DECIMAL(5,2),
        centuries INT,
        wickets INT,
        bowling_average DECIMAL(5,2),
        economy DECIMAL(5,2),
        PRIMARY KEY(player_id, format)
    );
    """
    cursor.execute(create_table_query)
    conn.commit()

    # --- Fetch all-rounder player IDs ---
    cursor.execute("SELECT player_id FROM players WHERE role LIKE '%Allrounder%'")
    player_ids = [row[0] for row in cursor.fetchall()]

    headers = {
        "x-rapidapi-key": "3eaece787amshfe1adf78b14d67bp1c781fjsnebfd830416ef",
        "x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
    }

    # --- Safe conversion functions ---
    def safe_int(x):
        try:
            return int(x)
        except:
            return None

    def safe_float(x):
        try:
            return float(x)
        except:
            return None

    # --- Process each player ---
    for player_id in player_ids:
        try:
            # Fetch batting stats
            bat_url = f"https://cricbuzz-cricket.p.rapidapi.com/stats/v1/player/{player_id}/batting"
            bat_data = requests.get(bat_url, headers=headers).json()

            # Skip if data missing
            if 'appIndex' not in bat_data or 'values' not in bat_data:
                print(f"Skipping player {player_id} – no batting data")
                continue

            # Fetch bowling stats
            bowl_url = f"https://cricbuzz-cricket.p.rapidapi.com/stats/v1/player/{player_id}/bowling"
            bowl_data = requests.get(bowl_url, headers=headers).json()

            if 'values' not in bowl_data:
                print(f"Skipping player {player_id} – no bowling data")
                continue

            # Player name
            player_name = bat_data.get('appIndex', {}).get('seoTitle', '').replace(" Profile", "")

            headers_list = bat_data['headers']  # ["ROWHEADER","Test","ODI","T20","IPL"]
            bat_values = bat_data['values']
            bowl_values = bowl_data['values']

            # --- Loop through each format ---
            for col_index in range(1, len(headers_list)):
                format_name = headers_list[col_index]

                # Batting stats
                runs = avg = hundreds = None
                for row in bat_values:
                    if row['values'][0] == "Runs":
                        runs = safe_int(row['values'][col_index])
                    elif row['values'][0] == "Average":
                        avg = safe_float(row['values'][col_index])
                    elif row['values'][0] == "100s":
                        hundreds = safe_int(row['values'][col_index])

                # Bowling stats
                wickets = bowl_avg = econ = None
                for row in bowl_values:
                    if row['values'][0] == "Wickets":
                        wickets = safe_int(row['values'][col_index])
                    elif row['values'][0] == "Avg":
                        bowl_avg = safe_float(row['values'][col_index])
                    elif row['values'][0] == "Eco":
                        econ = safe_float(row['values'][col_index])

                # --- Insert into MySQL ---
                player_tuple = (player_id, player_name, format_name, runs, avg, hundreds, wickets, bowl_avg, econ)
                insert_query = """
                INSERT INTO all_rounders_stats
                (player_id, player_name, format, runs, batting_average, centuries, wickets, bowling_average, economy)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
                ON DUPLICATE KEY UPDATE
                player_name=VALUES(player_name),
                runs=VALUES(runs),
                batting_average=VALUES(batting_average),
                centuries=VALUES(centuries),
                wickets=VALUES(wickets),
                bowling_average=VALUES(bowling_average),
                economy=VALUES(economy);
                """
                cursor.execute(insert_query, player_tuple)

            # Commit once per player
            conn.commit()
            print(f"Inserted stats for player: {player_name}")

        except Exception as e:
            print(f"Error for player {player_id}: {e}")

    cursor.close()
    conn.close()


In [2]:
all_rounder_stats()

Inserted stats for player: Ragheb Aga - Cricbuzz | Cricbuzz.com
Inserted stats for player: Sean Williams - Cricbuzz | Cricbuzz.com
Inserted stats for player: Collins Obuya - Cricbuzz | Cricbuzz.com
Inserted stats for player: Nehemiah Odhiambo - Cricbuzz | Cricbuzz.com
Inserted stats for player: Ravindra Jadeja - Cricbuzz | Cricbuzz.com
Inserted stats for player: Paul Stirling - Cricbuzz | Cricbuzz.com
Inserted stats for player: Mahmudullah - Cricbuzz | Cricbuzz.com
Inserted stats for player: Harvir Baidwan - Cricbuzz | Cricbuzz.com
Inserted stats for player: Seren Waters - Cricbuzz | Cricbuzz.com
Inserted stats for player: Angelo Mathews - Cricbuzz | Cricbuzz.com
Inserted stats for player: Mohammad Nabi - Cricbuzz | Cricbuzz.com
Inserted stats for player: Mitchell Marsh - Cricbuzz | Cricbuzz.com
Inserted stats for player: George Dockrell - Cricbuzz | Cricbuzz.com
Inserted stats for player: Nelson Odhiambo - Cricbuzz | Cricbuzz.com
Inserted stats for player: Ben Stokes - Cricbuzz | Cric